In [ ]:
# -------------------------x-----------------------x---------------------------x---------------------------x---------------------------x---------------------------
# Library Import|
# -------------------------x-----------------------x---------------------------x---------------------------x---------------------------x---------------------------


print('hello')
import pandas as pd


import datetime
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
import pandas_ta as pta
from datetime import datetime , timedelta , date , time
import MetaTrader5 as mt5 
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import math
import requests
import io
import plotly.express as px
import plotly.graph_objs as go
import plotly
import os
import glob
from calendar import monthrange



# -------------------------x-----------------------x---------------------------x---------------------------x---------------------------x---------------------------
# Option Back testing BuiltIn Function
# -------------------------x-----------------------x---------------------------x---------------------------x---------------------------x---------------------------

HolidayDateList = [date(2021,1,26),date(2021,3,11), date(2021,3,29), date(2021,4,2), date(2021,4,14), date(2021,4,21), date(2021,5,13), \
    date(2021,7,21), date(2021,8,19), date(2021,9,10), date(2021,10,15), date(2021,11,4) , date(2021,11,5) , date(2021,11,19),\
        date(2022,1,26),date(2022,3,1),date(2022,3,18), date(2022,4,14) , date(2022,4,15) , date(2022,5,3) , date(2022,8,9) , date(2022,8,15),\
            date(2022,8,31) , date(2022,10,5),date(2022,10,24) , date(2022,10,26),date(2022,11,8),\
                date(2020,2,21),date(2020,3,10),date(2020,4,2),date(2020,4,6),date(2020,4,10),date(2020,4,14),date(2020,5,1),date(2020,5,25),\
                    date(2020,10,2),date(2020,11,16),date(2020,11,30),date(2020,12,25)]



'''
Description - Convert Month Number to String Format

Input - MonthNumber (datatype - int)

Returns - MonthString (datatype - str )
'''
def numbers_to_strings(argument):
    switcher = {
        1: "JAN",
        2: "FEB",
        3: "MAR",
        4: "APR",
        5: "MAY",
        6: "JUN",
        7: "JUL",
        8: "AUG",
        9:  "SEP",
        10: "OCT",
        11: "NOV",
        12: "DEC"
        
    
    }
    return switcher.get(argument, "nothing")





'''
Description - Function to Clean the Option Data

Input - Unclean OptionData (datatype - Pandas DataFrame)

Steps Included
- Change The Columns Names.
- Create a datetime column by merging the date and time columns.
- Convert string to datetime for the datetime column in option data.
- Drop the date and Time Column.
- Drop the ContractName Column also.

Returns - Clean Option Data (datatype - Pandas DataFrame )
'''
def data_clean(OptionData):
    try:
        OptionData.columns = ['symbol','datetime', 'time', 'open', 'high', 'low', 'close', 'volume','interest']
        OptionData = OptionData.drop(index=0)
        OptionData['datetime'] = pd.to_datetime(OptionData['datetime'], format='%Y%m%d', errors='coerce')
        OptionData.dropna(subset=['datetime'], inplace=True)

        OptionData['time'] = pd.to_datetime(OptionData['time'], format='%H:%M:%S').dt.time
        # Convert the time column to a string
        OptionData['time'] = OptionData['time'].astype(str)

        # Convert the time column to a timedelta object
        OptionData['time'] = pd.to_timedelta(OptionData['time'])
        OptionData['datetime'] = OptionData['datetime'] + (OptionData['time'])
        OptionData.drop(['symbol'],axis = 1, inplace = True)
        OptionData.drop(['time'],axis = 1, inplace = True)
        OptionData.set_index('datetime', inplace=True)
        
        
    except ValueError:
        OptionData.columns = ['datetime', 'time', 'open', 'high', 'low', 'close', 'volume','interest']
        OptionData = OptionData.drop(index=0)
        OptionData['datetime'] = pd.to_datetime(OptionData['datetime'], format='%Y%m%d', errors='coerce')
        OptionData.dropna(subset=['datetime'], inplace=True)
        
        OptionData['time'] = pd.to_datetime(OptionData['time'], format='%H:%M:%S').dt.time
        # Convert the time column to a string
        OptionData['time'] = OptionData['time'].astype(str)

        # Convert the time column to a timedelta object
        OptionData['time'] = pd.to_timedelta(OptionData['time'])
        OptionData['datetime'] = OptionData['datetime'] + (OptionData['time'])
        OptionData.drop(['time'],axis = 1, inplace = True)
        OptionData.set_index('datetime', inplace=True)
    return OptionData
    
    
    
'''
Description - Function to Get a Option Contract on a Particular Time by passing the Price Parameter
This Function basically Run a Loop Through all the available contract of a particular expiry and then find the contract that is nearest to the price we have given.

Input - Price , CurrentTime , ParticularExpiry , Type(either CE or PE) , Path Of the Yearly Option Data Folder (datatype - float , datetime , str , str , str)

Steps Included :- 
- Get all the Files of a Contract for a Particular Year.
- Sort the files and Get the Contract with a particular Expiry that we have given and also Sort the CE or PE contract given by the user.
- Now loop through all the sorted files and find the price at the particular time we have given.
- Now Find the least distance between the price we have given and list of the price of different contract.
- Return the ContractName with the Least distance with the price we have given

Returns - ContractName , ContractPrice (datatype - str , float )
'''
def getContractBySP(Price,Time,Expiry,Type,path , LastThurs):
    
    ContractName = None
    ContractPrice = None
    files = glob.glob(path + "/*.csv")
    NewFiles = []
    for file in files:
        if LastThurs == False:
            if file.find(Expiry+Type)>=0:
                #print(file)
                NewFiles.append(file)
                
            elif file[-13:-4] == '00'+Expiry+Type:
                NewFiles.append(file)
    #for i in range()
    if Type == 'CE':
        NewFiles.sort(reverse=True)
    else:
        NewFiles.sort(reverse=False)
    dist = 1000
    for file in NewFiles:
        try:
            data = pd.read_csv(file)
            #print(data)
            df = data_clean(data.copy())
            #print(df)
            df = df.loc[Time:]
            if df.index[0].date() != Time.date():
                continue
            if df.index[0] != Time:
                continue
            #print(df)
            if abs(Price-df.iloc[0]['close']) < dist:
                ContractName = data.iloc[0]['Ticker']
                ContractPrice = df.iloc[0]['close']
                dist = abs(Price-df.iloc[0]['close'])
                pass
                #print(ContractName,ContractPrice,dist)

              
                
        except Exception as e:
            pass
        
        
        
    return ContractName , ContractPrice


'''
Description - Function to Get a Option Contract on a Particular Time that is greater than Price we have given
This Function basically Run a Loop Through all the available contract of a particular expiry and then find the contract that is greater than the price we have given.

Input - Price , CurrentTime , ParticularExpiry , Type(either CE or PE) , Path Of the Yearly Option Data Folder (datatype - float , datetime , str , str , str)

Steps Included :- 
- Get all the Files of a Contract for a Particular Year.
- Sort the files and Get the Contract with a particular Expiry that we have given.
- Now loop through all the sorted files and find the price at the particular time we have given.
- Now Find the ContractName that is nearest and greater than the Price we have given
- Return the ContractName with the Least distance with the price we have given

Returns - ContractName , ContractPrice (datatype - str , float )
'''

def getContractGreaterThanSP(Price,Time,Expiry,Type,path):
    
    ContractName = None
    ContractPrice = None
    files = glob.glob(path + "/*.csv")
    NewFiles = []
    for file in files:
        if file.find(Expiry+Type)>=0:
            NewFiles.append(file)

    TotalContract = pd.DataFrame(columns=['CN','CP'])
    for file in NewFiles:
        try:
            data = pd.read_csv(file)
            #print(data)
            df = data_clean(data.copy())
            #print(df)
            df = df.loc[Time:]
            if df.index[0].date() != Time.date():
                continue
            #print(df)
            
            TotalContract.loc[len(TotalContract)] = [data.iloc[0]['Ticker'], df.iloc[0]['close']]
                
        except Exception as e:
            pass
        
    df = TotalContract.sort_values(by='CP').reset_index(drop=True)
    for i in range(len(df)):
        if df.iloc[i]['CP'] > Price:
            ContractName = df.iloc[i]['CN']
            ContractPrice = df.iloc[i]['CP']
            break
        
    return ContractName , ContractPrice



'''
Description - Function to Check IF the Given Date is Last thursday of the month
Input - CurrentTime , OptionFrontWeekExpiry (datatype - datetime , datetime)

Steps Included :- 
- Import calendar library
- find out the last thursday on that month of CurrentTime
- If the day of this lastThursday is matching with Expiry's day then return True else False

Returns - True or False (datatype - boolean )
'''

def checkLastThursday(now,Expiry):
    import calendar
    last_thursday = max(week[-4] for week in calendar.monthcalendar(now.year,
                                                              now.month))
    if last_thursday==Expiry.day:
        return True
    else:
        return False
    
    
    
'''
Description - Function to Change the String Expiry of BankNifty option if that thursday is Holiday
Input - String Expiry  (datatype - str)

Steps Included :- 
- Create a Dictionary Switcher 
- Then use get function to get particular expiry fir that day
Returns - Original String Expiry (datatype - str )
'''
    
def HolidaySwitch(Date):
    switcher = {
        '02APR20' : '01APR20',
        '11MAR21' : '10MAR21',
        '13MAY21' : '12MAY21',
        '19AUG21':'18AUG21',
        '04NOV21' : '03NOV21',
        '14APR22' : '13APR22'
        
    }
    return switcher.get(Date,'nothing')



    

def getSameContractDay(today):
    if today.day == 1:
        TotalMonthDays = monthrange(today.year,today.month-1)[-1]
    else:
        TotalMonthDays = monthrange(today.year,today.month)[-1]
    dayValue = today.day
    today = today - timedelta(1)
    thursday = today+ timedelta((3-today.weekday()) % 7)
    thursday = thursday.day
    
    if TotalMonthDays == 31:
        if thursday == 31:
            thursday = 0
    elif TotalMonthDays == 30:
        if thursday == 30:
            thursday = 0
            
    elif TotalMonthDays == 29:
        if thursday == 29:
            thursday = 0
    elif TotalMonthDays == 28:
        if thursday == 28:
            thursday = 0
            
            
                    
            
    if TotalMonthDays == 31:
        if dayValue != 31:
            result = thursday+1
        else:
            result = 1

    elif TotalMonthDays == 30:
        if dayValue != 30:
            result = thursday + 1
        else:
            result = 1
    elif TotalMonthDays == 29:
        if dayValue != 29:
            result = thursday + 1
        else:
            return 1
    elif TotalMonthDays == 28:
        if dayValue != 28:
            result = thursday + 1
        else:
            result = 1
            
    return result
        
        
'''
Description - Find The Next Month Monthly BankNifty Expiry 
Input - CurrentTime (datatype - datetime)

Steps Included :- 
- Import Calendar
- If Current Month is not equal to 12 then find the last thursday from the calendar of next month
- If Current Month is equal to 12 then find the last thursday from the calendar of next month of the Next Year
Returns - NextMonthly Expiry (datatype - datetime )
'''
    
def FindNextMonthExpiry(today):
    import calendar
    
    if today.month != 12:
        last_thursday = max(week[-4] for week in calendar.monthcalendar(today.year,today.month+1))
        return datetime(today.year , today.month+1, last_thursday,15,15 )
    else:
        last_thursday = max(week[-4] for week in calendar.monthcalendar(today.year+1,1))
        return datetime(today.year+1 , 1, last_thursday,15,15)
      



'''
Description - Check If given Expiry is Last Thursday or not
Input - CurrentTime , Expiry (datatype - datetime)

Steps Included :- 
- Get the Date of Last thursday of the Month of CurrentTime
- If Last thursday day is matching with Given Expiry day return True else False.

Returns - True or False (datatype - boolean )
'''

def checkLastThursdayWithExpiry(now,Expiry):
    import calendar
    last_thursday = max(week[-4] for week in calendar.monthcalendar(now.year,
                                                              now.month))
    if last_thursday==Expiry.day:
        return True
    else:
        return False
    




'''
Description - Function to Clean the Option Data(Backfilled Data Version)

Input - Unclean OptionData (datatype - Pandas DataFrame)

Steps Included
- Change The Columns Names.
- Create a datetime column by merging the date and time columns.
- Convert string to datetime for the datetime column in option data.
- Drop the date and Time Column.
- Drop the ContractName Column also.

Returns - Clean Option Data (datatype - Pandas DataFrame )
'''

def data_clean_backfilled(ce):
    try:
        ce.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','interest' , 'symbol']
        ce['datetime'] = pd.to_datetime(ce['datetime'])
        ce.dropna(subset=['datetime'], inplace=True)
     
        ce.drop(['symbol'],axis = 1, inplace = True)
        ce.set_index('datetime', inplace=True)
        
        
    except ValueError:
        ce.columns = ['datetime', 'time', 'open', 'high', 'low', 'close', 'volume','interest']
        ce = ce.drop(index=0)
        ce['datetime'] = pd.to_datetime(ce['datetime'], format='%Y%m%d', errors='coerce')
        ce.dropna(subset=['datetime'], inplace=True)
        
        ce['time'] = pd.to_datetime(ce['time'], format='%H:%M:%S').dt.time
        # Convert the time column to a string
        ce['time'] = ce['time'].astype(str)

        # Convert the time column to a timedelta object
        ce['time'] = pd.to_timedelta(ce['time'])
        ce['datetime'] = ce['datetime'] + (ce['time'])
        ce.drop(['time'],axis = 1, inplace = True)
        ce.set_index('datetime', inplace=True)
    return ce



'''
Description - Function to Get a Option Contract on a Particular Time by passing the Price Parameter (Backfilled Version)
This Function basically Run a Loop Through all the available contract of a particular expiry and then find the contract that is nearest to the price we have given.

Input - Price , CurrentTime , ParticularExpiry , Type(either CE or PE) , Path Of the Yearly Option Data Folder (datatype - float , datetime , str , str , str)

Steps Included :- 
- Get all the Files of a Contract for a Particular Year.
- Sort the files and Get the Contract with a particular Expiry that we have given and also Sort the CE or PE contract given by the user.
- Now loop through all the sorted files and find the price at the particular time we have given.
- Now Find the least distance between the price we have given and list of the price of different contract.
- Return the ContractName with the Least distance with the price we have given

Returns - ContractName , ContractPrice (datatype - str , float )
'''

    
def getContractBySP_backfilled(Price,Time,Expiry,Type,path , LastThurs):
    
    ContractName = None
    ContractPrice = None
    files = glob.glob(path + "/*.csv")
    NewFiles = []
    for file in files:
        
        if LastThurs == False:
            if file.find(Expiry+Type)>=0:
                #print(file)
                NewFiles.append(file)
                
        elif file[-13:-4] == '00'+Expiry+Type:
            NewFiles.append(file)
    
    #for i in range()
    if Type == 'CE':
        NewFiles.sort(reverse=True)
    else:
        NewFiles.sort(reverse=False)
    dist = 1000
    for file in NewFiles:
        try:
            data = pd.read_csv(file)
            #print(data)
            df = data_clean_backfilled(data.copy())
            #print(df)
            df = df.loc[Time:]
            if df.index[0].date() != Time.date():
                continue
            if df.index[0] != Time:
                continue
            #print(df)
            if abs(Price-df.iloc[0]['close']) < dist:
                ContractName = data.iloc[0]['Ticker']
                ContractPrice = df.iloc[0]['close']
                dist = abs(Price-df.iloc[0]['close'])
                pass
                #print(ContractName,ContractPrice,dist)

              
                
        except Exception as e:
            pass
        
        
        
    return ContractName , ContractPrice


'''
Description - Find Option SL TP Price based on some price in Dollar or Rupees 

Input - OptionEntryPrice , LotSize , SingleLotQuantity , Dollar Price (datatype - float , float , float , float)

Steps Included :- 
- TpPrice = OptionEntryPrice + ((Dollar Price/LotSize) / SingleLotQuantity)

Returns - Target Or SL Price (datatype - float )
'''

def get_Price_point(Price , LotSize , Quantity , Points):
    
    x = Price +  ((Points/LotSize)/Quantity)
    
    return x






# -------------------------x-----------------------x---------------------------x---------------------------x---------------------------x---------------------------
# Strategy Spot Data Import
# -------------------------x-----------------------x---------------------------x---------------------------x---------------------------x---------------------------


# data = pd.read_csv(r'C:\Users\vijay\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\BANKNIFTY.csv', header=None)   
# data = data.drop(index=0)
# data.columns = ['symbol','datetime', 'time', 'open', 'high', 'low', 'close', 'volume','interest']
# data['datetime'] = pd.to_datetime(data['datetime'], format='%Y%m%d')
# data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S').dt.time
# # Convert the time column to a string
# data['time'] = data['time'].astype(str)

# # Convert the time column to a timedelta object
# data['time'] = pd.to_timedelta(data['time'])
# data['datetime'] = data['datetime'] + (data['time'])
# #         #df.drop(columns= [ 'time'], inplace= True)
 

# #data['datetime'] = data['datetime'].map(lambda Date : datetime.strptime(Date , '%Y%m%d %H:%M'))
# data.drop(['symbol'], axis=1, inplace = True)
# data.drop(['time'],axis = 1, inplace = True)
# data.set_index('datetime', inplace=True)

# data



# For Kite Data M1

data = pd.read_csv(r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\BANKNIFTYKiteM1.csv', header=None)   
data = data.drop(index=0)
data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
data['datetime'] = pd.to_datetime(data['datetime'])
# Convert the time column to a string
data.set_index('datetime', inplace=True)
data['close'] = data['close'].apply( lambda dt :float(dt))
data['open'] = data['open'].apply( lambda dt :float(dt))
data['high'] = data['high'].apply( lambda dt :float(dt))
data['low'] = data['low'].apply( lambda dt :float(dt))
data



# VIx data
vix = pd.read_csv(r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\IndiaVIX.csv', header=None)   
vix = vix.drop(index=0)
vix.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
vix['datetime'] = pd.to_datetime(vix['datetime'])
# Convert the time column to a string
vix.set_index('datetime', inplace=True)
vix['close'] = vix['close'].apply( lambda dt :float(dt))
vix['open'] = vix['open'].apply( lambda dt :float(dt))
vix['high'] = vix['high'].apply( lambda dt :float(dt))
vix['low'] = vix['low'].apply( lambda dt :float(dt))
vix




# # For Kite Data Weekly

# dataW1 = pd.read_csv(r'C:\Users\soura\OneDrive\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\BANKNIFTYKiteWeekly.csv', header=None)   
# dataW1 = dataW1.drop(index=0)
# dataW1.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
# dataW1.drop(index = len(dataW1),inplace=True)
# dataW1['datetime'] = pd.to_datetime(dataW1['datetime'])
# dataW1.set_index('datetime', inplace=True)
# dataW1['close'] = dataW1['close'].apply( lambda dt :float(dt))
# dataW1['open'] = dataW1['open'].apply( lambda dt :float(dt))
# dataW1['high'] = dataW1['high'].apply( lambda dt :float(dt))
# dataW1['low'] = dataW1['low'].apply( lambda dt :float(dt))




# # For Kite Data Weekly

# dataM30 = pd.read_csv(r'C:\Users\soura\OneDrive\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\BANKNIFTYKiteM30.csv', header=None)   
# dataM30 = dataM30.drop(index=0)
# dataM30.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
# dataM30['datetime'] = pd.to_datetime(dataM30['datetime'])
# dataM30.set_index('datetime', inplace=True)
# dataM30['close'] = dataM30['close'].apply( lambda dt :float(dt))
# dataM30['open'] = dataM30['open'].apply( lambda dt :float(dt))
# dataM30['high'] = dataM30['high'].apply( lambda dt :float(dt))
# dataM30['low'] = dataM30['low'].apply( lambda dt :float(dt))



vix

hello


,open,high,low,close,volume
datetime,,,,,
2019-12-16 09:15:00,13.30,13.35,12.98,13.19,0
2019-12-16 09:16:00,13.19,13.26,12.99,13.01,0
2019-12-16 09:17:00,13.01,13.10,12.99,13.07,0
2019-12-16 09:18:00,13.06,13.11,13.05,13.07,0
2019-12-16 09:19:00,13.07,13.07,13.03,13.03,0
...,...,...,...,...,...
2023-05-31 15:25:00,11.91,11.92,11.89,11.92,0
2023-05-31 15:26:00,11.92,11.95,11.91,11.95,0
2023-05-31 15:27:00,11.97,12.02,11.97,12.00,0


### Base Model



In [ ]:
path = r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\2020'

HolidayList = ['02APR20', '11MAR21',
               '13MAY21', '19AUG21', '04NOV21', '14APR22']
NextHolidayList = ['09APR20', '18MAR21',
                   '20MAY21', '00AUG21', '13NOV21', '21APR22']
OrderBasketID = 0
Orders = pd.DataFrame(columns=['entry', 'type', 'entry_time', 'exittime','entry_value', 'exitvalue', 'exit_type', 'SP', 'dd', 'BasketID','ContractName','LS'])
LotSize = 25

today = datetime(2019, 12, 11, 0, 0, 0)


# Strating Date and Ending Date of the backtest
StartingDate = datetime(2020,3,5,9,30)
tdayTime = StartingDate
ENdTime = datetime(2022, 12, 28, 9, 30, 0)
#NoDataAvailableCt = ['12MAR20' , '19MAR20' , '01APR20','MAR20','04JUN20' , '16APR20']
NoDataAvailableCt = ['19MAR20' , 'MAR20' , '01APR20']

LastTradeEndDate = data.index[0] - timedelta(days=1)

DataMisses = 0

while True:
    IsHoliday = False
    tdayTime = tdayTime + timedelta(minutes=1)



    # Set The Starting Date
    if tdayTime <= StartingDate:
        continue


    # Set the back testing End date
    if tdayTime >= ENdTime:
        break
    
    # if tdayTime.date() in [date(2020,3,12) ] :
    #         continue
        
        
    if (tdayTime + timedelta(days=1)).date() in HolidayDateList and (tdayTime + timedelta(days=1)).weekday() == 3:
        IsHoliday = True
        
    
    if tdayTime.date()  in HolidayDateList:
        continue
    
    


    # To change the file path for different years
    if tdayTime >= datetime(2021, 1, 1, 9, 30, 0):
        path = r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\2021'
    if tdayTime >= datetime(2022, 1, 1, 9, 30, 0):
        path = r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\2022'
        
        
        
        
        
        
    EntryWeekDay = 3
    
    if IsHoliday == True:
        EntryWeekDay = 2
        
    LastThrus = False
    # Enter Only Next Month last contract on this month last thursday
    if ((tdayTime.hour== 10 )and (tdayTime.minute == 00)) and (tdayTime.weekday() == EntryWeekDay):     
        print(tdayTime)
       
            
        

        NextExpiry = tdayTime + timedelta(days = 2)
        NextExpiry = NextExpiry + timedelta((3-NextExpiry.weekday()) % 7)
        #NextExpiry = NextExpiry + timedelta(hours=5.25)
        Expiryyear = NextExpiry.year
        ExpiryMonth = NextExpiry.month
        ExpiryDay = NextExpiry.day
        
        if NextExpiry in HolidayDateList:
            NextExpiry = NextExpiry - timedelta(days=1)
        
        # get the string value of the Expiry
        # if Expiryday is in 1 Digit we will add 0 at first
        if len(str(ExpiryDay)) == 1:
            StrExpiryDay = '0'+str(ExpiryDay)
        else:
            StrExpiryDay = str(ExpiryDay)
        StrExpiryYear = str(Expiryyear)[-2:]
        StrExpiryMonth = numbers_to_strings(ExpiryMonth)
    


        # if the Expiry is the last Expiry of the month we will add 00 at first to find the Contract
        if checkLastThursdayWithExpiry(tdayTime,  NextExpiry):
            StrExpiry = StrExpiryMonth+StrExpiryYear
            LastThrus = True
        else:
            StrExpiry = StrExpiryDay+StrExpiryMonth+StrExpiryYear


        # For Switching the Expiry to Wednesday where Thrusday is a Holiday
        if StrExpiry in HolidayList:
            StrExpiry = HolidaySwitch(StrExpiry)
            NextExpiry = NextExpiry - timedelta(days=1)
            
        
        
        
        CMP = data.loc[tdayTime]['open']
            
       
        
        #Rounding 
        CMP = round(float(CMP)/100)*100
        print(StrExpiry)
        
        if StrExpiry in NoDataAvailableCt:
            continue
        OrderBasketID+=1
        print(LastThrus)
        
        Price = 100
        SELLCEActiveContract , SELLCEActiveContractPrice = getContractBySP(Price, tdayTime, StrExpiry,'CE',path  , LastThrus)
        SELLPEActiveContract , SELLPEActiveContractPrice = getContractBySP(Price, tdayTime, StrExpiry,'PE',path , LastThrus)
        print(SELLCEActiveContract)
        
        BuyPrice = 1/3* (Price)
   
        FirstBuyCEActiveContract , Buy1CEEntryPrice =  getContractBySP(BuyPrice, tdayTime, StrExpiry,'CE',path , LastThrus)
        FirstBuyPEActiveContract , Buy1PEEntryPrice =  getContractBySP(BuyPrice, tdayTime, StrExpiry,'PE',path , LastThrus)

        
        
        try:
            SellCE1 = pd.read_csv(f'{path}\{SELLCEActiveContract}.csv')
            SellCE1 = data_clean(SellCE1)
            SellCE1 = SellCE1.drop(['volume','interest'] , axis =1)
            SellPE1 = pd.read_csv(f'{path}\{SELLPEActiveContract}.csv')
            SellPE1 = data_clean(SellPE1)
            SellPE1 = SellPE1.drop(['volume','interest'] , axis =1)
            
            Buy1CE2 = pd.read_csv(f'{path}\{FirstBuyCEActiveContract}.csv')
            Buy1CE2 = data_clean(Buy1CE2)
            Buy1CE2 = Buy1CE2.drop(['volume','interest'] , axis =1)
            Buy1PE2 = pd.read_csv(f'{path}\{FirstBuyPEActiveContract}.csv')
            Buy1PE2 = data_clean(Buy1PE2)
            Buy1PE2 = Buy1PE2.drop(['volume','interest'] , axis =1)

            
            mdf1 = pd.merge_asof(SellCE1, SellPE1, on='datetime')
            mdf2 = pd.merge_asof(mdf1, Buy1CE2 , on='datetime')
            mdf3 = pd.merge_asof(mdf2, Buy1PE2 , on='datetime')
            CEPEdf = pd.merge_asof(mdf3, data , on='datetime')
            CEPEdf.set_index('datetime', inplace=True)
        except FileNotFoundError:
            print(f'Data is not available in either PE or CE at Data Preprocessing for {tdayTime}')
            continue

        CEPEdf.columns = [f'open{SELLCEActiveContract}', f'high{SELLCEActiveContract}', f'low{SELLCEActiveContract}', f'close{SELLCEActiveContract}',\
            f'open{SELLPEActiveContract}', f'high{SELLPEActiveContract}', f'low{SELLPEActiveContract}', f'close{SELLPEActiveContract}',\
                f'open{FirstBuyCEActiveContract}', f'high{FirstBuyCEActiveContract}',f'low{FirstBuyCEActiveContract}', f'close{FirstBuyCEActiveContract}',\
                    f'open{FirstBuyPEActiveContract}', f'high{FirstBuyPEActiveContract}',f'low{FirstBuyPEActiveContract}', f'close{FirstBuyPEActiveContract}',\
                                'openSpot', 'highSpot' , 'lowSpot' , 'closeSpot' , 'volumeSpot']
        
        
        
        
        # Code To SKip the Baskets where there is Nan Data
        try:
            sample1 = CEPEdf.dropna().loc[tdayTime:]
            if (sample1.index[0] - tdayTime).total_seconds() >= 60 * 60 * 3:
                raise Exception
        except Exception as e:
            print(f'Data is not available in either PE or CE  for {tdayTime}')
            print(f'First Data Available : {CEPEdf.dropna().loc[tdayTime:].index[:5]}')
            print(f'Last Data Available : {CEPEdf.dropna().loc[tdayTime:].index[-5:]}')
            continue
        
        #break
            
        CEPEdf = CEPEdf.loc[tdayTime:]
        CEPEdf = CEPEdf.dropna()
        
        
        
        SELLCEEntryPrice = CEPEdf.iloc[0][f'open{SELLCEActiveContract}']
        SELLPEEntryPrice = CEPEdf.iloc[0][f'open{SELLPEActiveContract}']
        
        Buy1CEEntryPrice = CEPEdf.iloc[0][f'open{FirstBuyCEActiveContract}']
        Buy1PEEntryPrice = CEPEdf.iloc[0][f'open{FirstBuyPEActiveContract}']
        
    
        
        CEEntryTime = CEPEdf.index[0]
        PEEntryTime = CEPEdf.index[0]
        
  
    
        Target = 2500
 
        print(f'Entry at {CEEntryTime} at a SP of {CMP}')
        print(f'Enter First Sell Series CE Contract : {SELLCEActiveContract} Price : {SELLCEEntryPrice} . PE Contract : {SELLPEActiveContract} Price : {SELLPEEntryPrice}')
        print(f'Enter First Buy Series CE Contract : {FirstBuyCEActiveContract} Price : {Buy1CEEntryPrice} . PE Contract : {FirstBuyPEActiveContract} Price : {Buy1PEEntryPrice}')
        
        j = 0
        c = 0
        flagCE = 'NA'
        flagPE = 'NA'
        flag = 0
        
        #break
        while True and j <= len(CEPEdf) -1 :
            
            
            # get The Total Sum Price of all the Open Positions
            SellPnl = 25 * ((SELLCEEntryPrice - CEPEdf.iloc[j][f'close{SELLCEActiveContract}']) + (SELLPEEntryPrice - CEPEdf.iloc[j][f'close{SELLPEActiveContract}']))
            BuyFirstPnl = 25 * ((CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'] - Buy1CEEntryPrice) + (CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'] - Buy1PEEntryPrice) )
            TotalPnl = SellPnl + BuyFirstPnl 
            
            if flag == 1:
                TotalPrice = CEPEdf.iloc[j][f'close{SELLCEActiveContract}'] + CEPEdf.iloc[j][f'close{SELLPEActiveContract}']
                
                if TotalPrice > StopLoss:
                
                    print(f'StopLoss Hit at {CEPEdf.index[j]} .Closed Sell CE Contract : {SELLCEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLCEActiveContract}"]} .Closed Sell PE Contract : {SELLPEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLPEActiveContract}"]}')
                    print(f'Closed Buy CE Contract : {FirstBuyCEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyCEActiveContract}"]} .Closed Buy PE Contract : {FirstBuyPEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyPEActiveContract}"]} \n\n\n\n')
                    
                    # Exit FIrst SELL CE Order
                    try:
                        ddCt = SELLPEEntryPrice - max(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [-1 ,  'CE',  CEEntryTime, CEPEdf.index[j] ,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, SELLCEActiveContract ,1]
                    
                    # Exit First SELL PE Order
                    try:
                        ddCt = SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [-1 ,  'PE',  PEEntryTime, CEPEdf.index[j] ,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, SELLPEActiveContract,1 ]
                    
        
                    
                    # Exit FirstBuy CE Order
                    try:
                        ddCt = min(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [1 ,  'CE',  CEEntryTime, CEPEdf.index[j] ,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, FirstBuyCEActiveContract ,1]
                    
                    
                    # Exit FirstBuy PE Order
                    try:
                        ddCt = min(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [1 ,  'PE',  PEEntryTime, CEPEdf.index[j] ,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, FirstBuyPEActiveContract , 1]
                    
            
                    break
            
            
            
            ## CE Adjustment
            if flag == 0 and (0.5 * CEPEdf.iloc[j][f'close{SELLPEActiveContract}']) >= CEPEdf.iloc[j][f'close{SELLCEActiveContract}']:
                AdjustedSELLPrice = 0.8 * CEPEdf.iloc[j][f'close{SELLPEActiveContract}']
                
                AdjustmentTime = CEPEdf.index[j]
                print(f'Adjustment of CE at {AdjustmentTime} at price :{AdjustedSELLPrice}')
                if AdjustmentTime == datetime( 2020,3,6,9,17):
                    print('hello')
                 # Updating the Orders Data frame
                 
                # Sell Order
                try:
                    ddCE = (SELLCEEntryPrice - max(CEPEdf.loc[CEEntryTime : AdjustmentTime][f'high{SELLCEActiveContract}']))
                except ValueError:
                    ddCE = 0
                # ['entry', 'type', 'entry_time', 'exittime','entry_value', 'exitvalue', 'exit_type', 'SP', 'dd', 'BasketID','ContractName','LS']
                Orders.loc[len(Orders)] = [-1, 'CE', CEEntryTime, AdjustmentTime,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'Adj', CMP, ddCE, OrderBasketID ,SELLCEActiveContract , 1 ]
                
                # Buy Order
                try:
                    ddCE = (min(CEPEdf.loc[CEEntryTime : AdjustmentTime][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice)
                except ValueError:
                    ddCE = 0
                Orders.loc[len(Orders)] = [1, 'CE', CEEntryTime, AdjustmentTime,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'Adj', 'UP', ddCE, OrderBasketID, FirstBuyCEActiveContract, 1]
                print(f"Exited SELL Contract : {SELLCEActiveContract} at Price : {CEPEdf.iloc[j][f'close{SELLCEActiveContract}']} and NEW BUY Contract : {FirstBuyCEActiveContract} and Price : {CEPEdf.iloc[j][f'close{SELLCEActiveContract}']}")
                
                
                
                
                NewSELLCEActiveContract , NewSELLCEActiveContractPrice = getContractBySP(AdjustedSELLPrice , AdjustmentTime , StrExpiry , 'CE' , path  , LastThrus)
                AdjustedBUYPrice = 1/3 * NewSELLCEActiveContractPrice
                NewFirstBuyCEActiveContract , NewFirstBuyCEActiveContractPrice= getContractBySP(AdjustedBUYPrice , AdjustmentTime , StrExpiry , 'CE' , path , LastThrus )
                
                
                NewSELLCEdf = data_clean(pd.read_csv(f'{path}\{NewSELLCEActiveContract}.csv'))
                NewSELLCEdf = NewSELLCEdf.drop(['volume','interest'] , axis =1)
                NewBuyCEdf = data_clean(pd.read_csv(f'{path}\{NewFirstBuyCEActiveContract}.csv'))
                NewBuyCEdf = NewBuyCEdf.drop(['volume','interest'] , axis =1)
                
                NewColumns = ['datetime'] + list(CEPEdf.columns) + [f'open{NewSELLCEActiveContract}', f'high{NewSELLCEActiveContract}', f'low{NewSELLCEActiveContract}', f'close{NewSELLCEActiveContract}',\
                f'open{NewFirstBuyCEActiveContract}', f'high{NewFirstBuyCEActiveContract}',f'low{NewFirstBuyCEActiveContract}', f'close{NewFirstBuyCEActiveContract}']
                mdf1 = pd.merge_asof(CEPEdf, NewSELLCEdf, on='datetime')
                CEPEdf = pd.merge_asof(mdf1, NewBuyCEdf, on='datetime')
                CEPEdf.columns = NewColumns
                CEPEdf.set_index('datetime' , inplace=True)
                CEPEdf = CEPEdf.loc[:,~CEPEdf.columns.duplicated()]
                
                CEPEdf = CEPEdf.loc[AdjustmentTime:]
                j = 0
                
                SELLCEActiveContract = NewSELLCEActiveContract
                FirstBuyCEActiveContract = NewFirstBuyCEActiveContract
                SELLCEEntryPrice = CEPEdf.iloc[j][f'open{SELLCEActiveContract}']
                Buy1CEEntryPrice = CEPEdf.iloc[j][f'open{FirstBuyCEActiveContract}']
                print(f'Entered New SELL Contract : {SELLCEActiveContract} at Price : {SELLCEEntryPrice} and NEW BUY Contract : {FirstBuyCEActiveContract} and Price : {Buy1CEEntryPrice}')
                
                
                ## Stoploss Activation Condition:
                if (int(SELLPEActiveContract[9:14]) - int(SELLCEActiveContract[9:14])) >= -100:
                    StopLoss = (SELLCEEntryPrice + CEPEdf.iloc[j][f'open{SELLPEActiveContract}']) * 1.1
                    print(f'Stoploss Created at {CEPEdf.index[j]}')
                    
                    flag = 1
                
            
                CEEntryTime = AdjustmentTime
            
            
            
            ## PE Adjustment
            elif flag == 0 and (0.5 * CEPEdf.iloc[j][f'close{SELLCEActiveContract}']) >= CEPEdf.iloc[j][f'close{SELLPEActiveContract}']:
                AdjustedSELLPrice = 0.8 * CEPEdf.iloc[j][f'close{SELLCEActiveContract}']
                
                AdjustmentTime = CEPEdf.index[j]
                print(f'Adjustment of PE at {AdjustmentTime} price : {AdjustedSELLPrice}')
                
                 # Updating the Orders Data frame
                 
                # Sell Order
                try:
                    ddCE = (SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : AdjustmentTime][f'high{SELLPEActiveContract}']))
                except ValueError:
                    ddCE = 0
                Orders.loc[len(Orders)] = [-1, 'PE', PEEntryTime, AdjustmentTime,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'Adj', CMP, ddCE, OrderBasketID ,SELLPEActiveContract , 1 ]
                
                # Buy Order
                try:
                    ddCE = (min(CEPEdf.loc[PEEntryTime : AdjustmentTime][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice)
                except ValueError:
                    ddCE = 0
                Orders.loc[len(Orders)] = [1, 'PE', PEEntryTime, AdjustmentTime,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'Adj', 'UP', ddCE, OrderBasketID, FirstBuyPEActiveContract, 1]
                print(f"Exited SELL Contract : {SELLPEActiveContract} at Price : {CEPEdf.iloc[j][f'close{SELLPEActiveContract}']} and NEW BUY Contract : {FirstBuyPEActiveContract} and Price : {CEPEdf.iloc[j][f'close{SELLPEActiveContract}']}")
                
                
                
                NewSELLPEActiveContract , NewSELLPEActiveContractPrice = getContractBySP(AdjustedSELLPrice , AdjustmentTime , StrExpiry , 'PE' , path ,LastThrus )
                AdjustedBUYPrice = 1/3 * NewSELLPEActiveContractPrice
                NewFirstBuyPEActiveContract , NewFirstBuyPEActiveContractPrice= getContractBySP(AdjustedBUYPrice , AdjustmentTime , StrExpiry , 'PE' , path, LastThrus )
                
                
                NewSELLPEdf = data_clean(pd.read_csv(f'{path}\{NewSELLPEActiveContract}.csv'))
                NewSELLPEdf = NewSELLPEdf.drop(['volume','interest'] , axis =1)
                NewBuyPEdf = data_clean(pd.read_csv(f'{path}\{NewFirstBuyPEActiveContract}.csv'))
                NewBuyPEdf = NewBuyPEdf.drop(['volume','interest'] , axis =1)
                
                NewColumns = ['datetime'] + list(CEPEdf.columns) + [f'open{NewSELLPEActiveContract}', f'high{NewSELLPEActiveContract}', f'low{NewSELLPEActiveContract}', f'close{NewSELLPEActiveContract}',\
                f'open{NewFirstBuyPEActiveContract}', f'high{NewFirstBuyPEActiveContract}',f'low{NewFirstBuyPEActiveContract}', f'close{NewFirstBuyPEActiveContract}']
                mdf1 = pd.merge_asof(CEPEdf, NewSELLPEdf, on='datetime')
                CEPEdf = pd.merge_asof(mdf1, NewBuyPEdf, on='datetime')
                CEPEdf.columns = NewColumns
                CEPEdf.set_index('datetime' , inplace=True)
                CEPEdf = CEPEdf.loc[:,~CEPEdf.columns.duplicated()]
                
                CEPEdf = CEPEdf.loc[AdjustmentTime:]
                j = 0
                
                SELLPEActiveContract = NewSELLPEActiveContract
                FirstBuyPEActiveContract = NewFirstBuyPEActiveContract
                SELLPEEntryPrice = CEPEdf.iloc[j][f'open{SELLPEActiveContract}']
                Buy1PEEntryPrice = CEPEdf.iloc[j][f'open{FirstBuyPEActiveContract}']
                print(f'Entered New SELL Contract : {SELLPEActiveContract} at Price : {SELLPEEntryPrice} and NEW BUY Contract : {FirstBuyPEActiveContract} and Price : {Buy1PEEntryPrice}')
                
                
                ## Stoploss Activation Condition:
                if (int(SELLPEActiveContract[9:14]) - int(SELLCEActiveContract[9:14])) >= -100:
                    StopLoss = (SELLCEEntryPrice + CEPEdf.iloc[j][f'open{SELLPEActiveContract}']) * 1.1
                    print(f'Stoploss Created at {CEPEdf.index[j]}')                    
                    flag = 1


                PEEntryTime = AdjustmentTime
            
            
            
            # BreakPoint Above Exit Condition
            if TotalPnl >= Target:
                
                print(f'Target Hit at {CEPEdf.index[j]} .Closed Sell CE Contract : {SELLCEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLCEActiveContract}"]} .Closed Sell PE Contract : {SELLPEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLPEActiveContract}"]}')
                print(f'Closed Buy CE Contract : {FirstBuyCEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyCEActiveContract}"]} .Closed Buy PE Contract : {FirstBuyPEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyPEActiveContract}"]} \n\n\n\n')
                
                # Exit FIrst SELL CE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  'CE',  CEEntryTime, CEPEdf.index[j] ,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, SELLCEActiveContract ,1]
                
                # Exit First SELL PE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  'PE',  PEEntryTime, CEPEdf.index[j] ,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, SELLPEActiveContract,1 ]
                
    
                
                # Exit FirstBuy CE Order
                try:
                    ddCt = min(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  'CE',  CEEntryTime, CEPEdf.index[j] ,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, FirstBuyCEActiveContract ,1]
                
                
                # Exit FirstBuy PE Order
                try:
                    ddCt = min(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  'PE',  PEEntryTime, CEPEdf.index[j] ,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, FirstBuyPEActiveContract , 1]
                
           
                break
            
            
        
            
    
       
            
          
            
            
            # Expiry Exit Condition
            if CEPEdf.index[j] >= NextExpiry:
                print(f'Expiry Exit at {CEPEdf.index[j]} .Closed Sell CE Contract : {SELLCEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLCEActiveContract}"]} .Closed Sell PE Contract : {SELLPEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLPEActiveContract}"]}')
                print(f'Closed Buy CE Contract : {FirstBuyCEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyCEActiveContract}"]} .Closed Buy PE Contract : {FirstBuyPEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyPEActiveContract}"]} \n\n\n\n')
                
                # Exit FIrst SELL CE Order
                # Exit FIrst SELL CE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  'CE',  CEEntryTime, CEPEdf.index[j] ,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, SELLCEActiveContract ,1]
                
                # Exit First SELL PE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  'PE',  PEEntryTime, CEPEdf.index[j] ,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, SELLPEActiveContract,1 ]
                
               
                
                # Exit FirstBuy CE Order
                try:
                    ddCt = min(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  "CE",  CEEntryTime, CEPEdf.index[j] ,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, FirstBuyCEActiveContract ,1]
                
                
                # Exit FirstBuy PE Order
                try:
                    ddCt = min(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  'PE',  PEEntryTime, CEPEdf.index[j] ,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, FirstBuyPEActiveContract , 1]
              
               
                break
               
            
            
        
        

    
            j+=1
                
        # Break For 1 Trade
        break
                    
                

            

2020-03-05 10:00:00
12MAR20
False
BANKNIFTY2960012MAR20CE
Entry at 2020-03-05 10:00:00 at a SP of 28600
Enter First Sell Series CE Contract : BANKNIFTY2960012MAR20CE Price : 97.95 . PE Contract : BANKNIFTY2740012MAR20PE Price : 118.75
Enter First Buy Series CE Contract : BANKNIFTY3020012MAR20CE Price : 34.2 . PE Contract : BANKNIFTY2650012MAR20PE Price : 37.5
Adjustment of PE at 2020-03-05 10:46:00 price : 113.68
Exited SELL Contract : BANKNIFTY2740012MAR20PE at Price : 70.0 and NEW BUY Contract : BANKNIFTY2650012MAR20PE and Price : 70.0
Entered New SELL Contract : BANKNIFTY2780012MAR20PE at Price : 120.0 and NEW BUY Contract : BANKNIFTY2700012MAR20PE and Price : 47.6
Adjustment of PE at 2020-03-05 11:30:00 price : 161.44000000000003
Exited SELL Contract : BANKNIFTY2780012MAR20PE at Price : 100.55 and NEW BUY Contract : BANKNIFTY2700012MAR20PE and Price : 100.55
Entered New SELL Contract : BANKNIFTY2820012MAR20PE at Price : 168.4 and NEW BUY Contract : BANKNIFTY2740012MAR20PE and Price

In [10]:
Orders

,entry,type,entry_time,exittime,entry_value,exitvalue,exit_type,SP,dd,BasketID,ContractName,LS
0,-1,PE,2020-01-02 10:00:00,2020-01-02 12:14:00,94.90,71.85,Adj,32200,-6.35,1,BANKNIFTY3190009JAN20PE,1
1,1,PE,2020-01-02 10:00:00,2020-01-02 12:14:00,30.50,23.45,Adj,UP,-7.25,1,BANKNIFTY3150009JAN20PE,1
2,-1,PE,2020-01-02 12:14:00,2020-01-02 14:23:00,126.75,92.90,Adj,32200,-13.25,1,BANKNIFTY3210009JAN20PE,1
3,1,PE,2020-01-02 12:14:00,2020-01-02 14:23:00,42.20,31.25,Adj,UP,-10.95,1,BANKNIFTY3170009JAN20PE,1
4,-1,CE,2020-01-02 10:00:00,2020-01-03 09:30:00,108.55,119.00,Adj,32200,-108.45,1,BANKNIFTY3250009JAN20CE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1371,1,CE,2021-12-24 09:42:00,2021-12-27 09:15:00,75.20,23.10,Adj,UP,-53.15,101,BANKNIFTY35900DEC21CE,1
1372,-1,BANKNIFTY34200DEC21CE,2021-12-27 09:15:00,2021-12-27 09:16:00,627.15,490.35,SL,35200,-431.25,101,BANKNIFTY34200DEC21CE,1
1373,-1,BANKNIFTY34800DEC21PE,2021-12-24 14:15:00,2021-12-27 09:16:00,249.15,607.55,SL,35200,-431.25,101,BANKNIFTY34800DEC21PE,1
1374,1,BANKNIFTY34900DEC21CE,2021-12-27 09:15:00,2021-12-27 09:16:00,344.00,158.55,SL,35200,-202.00,101,BANKNIFTY34900DEC21CE,1


### Base Model Backfilled


In [2]:
path = r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionBackFilled\2020_new'

HolidayList = ['02APR20', '11MAR21',
               '13MAY21', '19AUG21', '04NOV21', '14APR22']
NextHolidayList = ['09APR20', '18MAR21',
                   '20MAY21', '00AUG21', '13NOV21', '21APR22']
OrderBasketID = 0
Orders = pd.DataFrame(columns=['entry', 'type', 'entry_time', 'exittime','entry_value', 'exitvalue', 'exit_type', 'SP', 'dd', 'BasketID','ContractName','LS'])
LotSize = 25

today = datetime(2019, 12, 11, 0, 0, 0)


# Strating Date and Ending Date of the backtest
StartingDate = datetime(2020,1,1,9,30)
tdayTime = StartingDate
ENdTime = datetime(2021, 12, 28, 9, 30, 0)
#NoDataAvailableCt = ['12MAR20' , '19MAR20' , '01APR20','MAR20','04JUN20' , '16APR20']
NoDataAvailableCt = ['19MAR20' , 'MAR20' , '01APR20']

LastTradeEndDate = data.index[0] - timedelta(days=1)

DataMisses = 0

while True:
    IsHoliday = False
    tdayTime = tdayTime + timedelta(minutes=1)



    # Set The Starting Date
    if tdayTime <= StartingDate:
        continue


    # Set the back testing End date
    if tdayTime >= ENdTime:
        break
    
    # if tdayTime.date() in [date(2020,3,12) ] :
    #         continue
        
        
    if (tdayTime + timedelta(days=1)).date() in HolidayDateList and (tdayTime + timedelta(days=1)).weekday() == 3:
        IsHoliday = True
        
    
    if tdayTime.date()  in HolidayDateList:
        continue
    
    


    # To change the file path for different years
    if tdayTime >= datetime(2021, 1, 1, 9, 30, 0):
        path = r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionBackFilled\2021_new'
    if tdayTime >= datetime(2022, 1, 1, 9, 30, 0):
        path = r'C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionStrategy\IndianOptionData\2022_new'
        
        
        
        
        
        
    EntryWeekDay = 3
    
    if IsHoliday == True:
        EntryWeekDay = 2
        
    LastThrus = False
    # Enter Only Next Month last contract on this month last thursday
    if ((tdayTime.hour== 10 )and (tdayTime.minute == 00)) and (tdayTime.weekday() == EntryWeekDay):     
        print(tdayTime)
       
            
        

        NextExpiry = tdayTime + timedelta(days = 2)
        NextExpiry = NextExpiry + timedelta((3-NextExpiry.weekday()) % 7)
        #NextExpiry = NextExpiry + timedelta(hours=5.25)
        Expiryyear = NextExpiry.year
        ExpiryMonth = NextExpiry.month
        ExpiryDay = NextExpiry.day
        
        if NextExpiry in HolidayDateList:
            NextExpiry = NextExpiry - timedelta(days=1)
        
        # get the string value of the Expiry
        # if Expiryday is in 1 Digit we will add 0 at first
        if len(str(ExpiryDay)) == 1:
            StrExpiryDay = '0'+str(ExpiryDay)
        else:
            StrExpiryDay = str(ExpiryDay)
        StrExpiryYear = str(Expiryyear)[-2:]
        StrExpiryMonth = numbers_to_strings(ExpiryMonth)
    


        # if the Expiry is the last Expiry of the month we will add 00 at first to find the Contract
        if checkLastThursdayWithExpiry(tdayTime,  NextExpiry):
            StrExpiry = StrExpiryMonth+StrExpiryYear
            LastThrus = True
        else:
            StrExpiry = StrExpiryDay+StrExpiryMonth+StrExpiryYear


        # For Switching the Expiry to Wednesday where Thrusday is a Holiday
        if StrExpiry in HolidayList:
            StrExpiry = HolidaySwitch(StrExpiry)
            NextExpiry = NextExpiry - timedelta(days=1)
            
        
        
        
        CMP = data.loc[tdayTime]['open']
            
       
        
        #Rounding 
        CMP = round(float(CMP)/100)*100
        print(StrExpiry)
        
        if StrExpiry in NoDataAvailableCt:
            continue
        OrderBasketID+=1
        print(LastThrus)
        
        Price = 100
        SELLCEActiveContract , SELLCEActiveContractPrice = getContractBySP_backfilled(Price, tdayTime, StrExpiry,'CE',path , LastThrus)
        SELLPEActiveContract , SELLPEActiveContractPrice = getContractBySP_backfilled(Price, tdayTime, StrExpiry,'PE',path, LastThrus)
        print(SELLCEActiveContract)
        
        BuyPrice = 1/3* (Price)
   
        FirstBuyCEActiveContract , Buy1CEEntryPrice =  getContractBySP_backfilled(BuyPrice, tdayTime, StrExpiry,'CE',path, LastThrus)
        FirstBuyPEActiveContract , Buy1PEEntryPrice =  getContractBySP_backfilled(BuyPrice, tdayTime, StrExpiry,'PE',path, LastThrus)

        
        
        try:
            SellCE1 = pd.read_csv(f'{path}\{SELLCEActiveContract}.csv')
            SellCE1 = data_clean_backfilled(SellCE1)
            SellCE1 = SellCE1.drop(['volume','interest'] , axis =1)
            SellPE1 = pd.read_csv(f'{path}\{SELLPEActiveContract}.csv')
            SellPE1 = data_clean_backfilled(SellPE1)
            SellPE1 = SellPE1.drop(['volume','interest'] , axis =1)
            
            Buy1CE2 = pd.read_csv(f'{path}\{FirstBuyCEActiveContract}.csv')
            Buy1CE2 = data_clean_backfilled(Buy1CE2)
            Buy1CE2 = Buy1CE2.drop(['volume','interest'] , axis =1)
            Buy1PE2 = pd.read_csv(f'{path}\{FirstBuyPEActiveContract}.csv')
            Buy1PE2 = data_clean_backfilled(Buy1PE2)
            Buy1PE2 = Buy1PE2.drop(['volume','interest'] , axis =1)

            
            mdf1 = pd.merge_asof(SellCE1, SellPE1, on='datetime')
            mdf2 = pd.merge_asof(mdf1, Buy1CE2 , on='datetime')
            mdf3 = pd.merge_asof(mdf2, Buy1PE2 , on='datetime')
            CEPEdf = pd.merge_asof(mdf3, data , on='datetime')
            CEPEdf.set_index('datetime', inplace=True)
        except FileNotFoundError:
            print(f'Data is not available in either PE or CE at Data Preprocessing for {tdayTime}')
            continue

        CEPEdf.columns = [f'open{SELLCEActiveContract}', f'high{SELLCEActiveContract}', f'low{SELLCEActiveContract}', f'close{SELLCEActiveContract}',\
            f'open{SELLPEActiveContract}', f'high{SELLPEActiveContract}', f'low{SELLPEActiveContract}', f'close{SELLPEActiveContract}',\
                f'open{FirstBuyCEActiveContract}', f'high{FirstBuyCEActiveContract}',f'low{FirstBuyCEActiveContract}', f'close{FirstBuyCEActiveContract}',\
                    f'open{FirstBuyPEActiveContract}', f'high{FirstBuyPEActiveContract}',f'low{FirstBuyPEActiveContract}', f'close{FirstBuyPEActiveContract}',\
                                'openSpot', 'highSpot' , 'lowSpot' , 'closeSpot' , 'volumeSpot']
        
        
        
        
        # Code To SKip the Baskets where there is Nan Data
        try:
            sample1 = CEPEdf.dropna().loc[tdayTime:]
            if (sample1.index[0] - tdayTime).total_seconds() >= 60 * 60 * 3:
                raise Exception
        except Exception as e:
            print(f'Data is not available in either PE or CE  for {tdayTime}')
            print(f'First Data Available : {CEPEdf.dropna().loc[tdayTime:].index[:5]}')
            print(f'Last Data Available : {CEPEdf.dropna().loc[tdayTime:].index[-5:]}')
            break
            continue
        
        #break
            
        CEPEdf = CEPEdf.loc[tdayTime:]
        CEPEdf = CEPEdf.dropna()
        
        
        
        SELLCEEntryPrice = CEPEdf.iloc[0][f'open{SELLCEActiveContract}']
        SELLPEEntryPrice = CEPEdf.iloc[0][f'open{SELLPEActiveContract}']
        
        Buy1CEEntryPrice = CEPEdf.iloc[0][f'open{FirstBuyCEActiveContract}']
        Buy1PEEntryPrice = CEPEdf.iloc[0][f'open{FirstBuyPEActiveContract}']
        
    
        
        CEEntryTime = CEPEdf.index[0]
        PEEntryTime = CEPEdf.index[0]
        
  
    
        Target = 2500
 
        print(f'Entry at {CEEntryTime} at a SP of {CMP}')
        print(f'Enter First Sell Series CE Contract : {SELLCEActiveContract} Price : {SELLCEEntryPrice} . PE Contract : {SELLPEActiveContract} Price : {SELLPEEntryPrice}')
        print(f'Enter First Buy Series CE Contract : {FirstBuyCEActiveContract} Price : {Buy1CEEntryPrice} . PE Contract : {FirstBuyPEActiveContract} Price : {Buy1PEEntryPrice}')
        
        j = 0
        c = 0
        flagCE = 'NA'
        flagPE = 'NA'
        flag = 0
        
        #break
        while True and j <= len(CEPEdf) -1 :
            
            
            # get The Total Sum Price of all the Open Positions
            SellPnl = 25 * ((SELLCEEntryPrice - CEPEdf.iloc[j][f'close{SELLCEActiveContract}']) + (SELLPEEntryPrice - CEPEdf.iloc[j][f'close{SELLPEActiveContract}']))
            BuyFirstPnl = 25 * ((CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'] - Buy1CEEntryPrice) + (CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'] - Buy1PEEntryPrice) )
            TotalPnl = SellPnl + BuyFirstPnl 
            
            if flag == 1:
                TotalPrice = CEPEdf.iloc[j][f'close{SELLCEActiveContract}'] + CEPEdf.iloc[j][f'close{SELLPEActiveContract}']
                
                if TotalPrice > StopLoss:
                
                    print(f'StopLoss Hit at {CEPEdf.index[j]} .Closed Sell CE Contract : {SELLCEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLCEActiveContract}"]} .Closed Sell PE Contract : {SELLPEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLPEActiveContract}"]}')
                    print(f'Closed Buy CE Contract : {FirstBuyCEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyCEActiveContract}"]} .Closed Buy PE Contract : {FirstBuyPEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyPEActiveContract}"]} \n\n\n\n')
                    
                    # Exit FIrst SELL CE Order
                    try:
                        ddCt = SELLPEEntryPrice - max(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [-1 ,  SELLCEActiveContract,  CEEntryTime, CEPEdf.index[j] ,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, SELLCEActiveContract ,1]
                    
                    # Exit First SELL PE Order
                    try:
                        ddCt = SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [-1 ,  SELLPEActiveContract,  PEEntryTime, CEPEdf.index[j] ,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, SELLPEActiveContract,1 ]
                    
        
                    
                    # Exit FirstBuy CE Order
                    try:
                        ddCt = min(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [1 ,  FirstBuyCEActiveContract,  CEEntryTime, CEPEdf.index[j] ,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, FirstBuyCEActiveContract ,1]
                    
                    
                    # Exit FirstBuy PE Order
                    try:
                        ddCt = min(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice
                    except ValueError:
                        ddCt = 0
                    Orders.loc[len(Orders)] = [1 ,  FirstBuyPEActiveContract,  PEEntryTime, CEPEdf.index[j] ,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'SL', CMP, ddCt , OrderBasketID, FirstBuyPEActiveContract , 1]
                    
            
                    break
            
            
            
            ## CE Adjustment
            if flag == 0 and (0.5 * CEPEdf.iloc[j][f'close{SELLPEActiveContract}']) >= CEPEdf.iloc[j][f'close{SELLCEActiveContract}']:
                AdjustedSELLPrice = 0.8 * CEPEdf.iloc[j][f'close{SELLPEActiveContract}']
                
                AdjustmentTime = CEPEdf.index[j]
                print(f'Adjustment of CE at {AdjustmentTime}')
                
                 # Updating the Orders Data frame
                 
                # Sell Order
                try:
                    ddCE = (SELLCEEntryPrice - max(CEPEdf.loc[CEEntryTime : AdjustmentTime][f'high{SELLCEActiveContract}']))
                except ValueError:
                    ddCE = 0
                # ['entry', 'type', 'entry_time', 'exittime','entry_value', 'exitvalue', 'exit_type', 'SP', 'dd', 'BasketID','ContractName','LS']
                Orders.loc[len(Orders)] = [-1, 'CE', CEEntryTime, AdjustmentTime,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'Adj', CMP, ddCE, OrderBasketID ,SELLCEActiveContract , 1 ]
                
                # Buy Order
                try:
                    ddCE = (min(CEPEdf.loc[CEEntryTime : AdjustmentTime][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice)
                except ValueError:
                    ddCE = 0
                Orders.loc[len(Orders)] = [1, 'CE', CEEntryTime, AdjustmentTime,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'Adj', 'UP', ddCE, OrderBasketID, FirstBuyCEActiveContract, 1]
                print(f"Exited SELL Contract : {SELLCEActiveContract} at Price : {CEPEdf.iloc[j][f'close{SELLCEActiveContract}']} and NEW BUY Contract : {FirstBuyCEActiveContract} and Price : {CEPEdf.iloc[j][f'close{SELLCEActiveContract}']}")
                
                
                
                
                NewSELLCEActiveContract , NewSELLCEActiveContractPrice = getContractBySP_backfilled(AdjustedSELLPrice , AdjustmentTime , StrExpiry , 'CE' , path , LastThrus)
                AdjustedBUYPrice = 1/3 * NewSELLCEActiveContractPrice
                NewFirstBuyCEActiveContract , NewFirstBuyCEActiveContractPrice= getContractBySP_backfilled(AdjustedBUYPrice , AdjustmentTime , StrExpiry , 'CE' , path , LastThrus)
                
                
                NewSELLCEdf = data_clean_backfilled(pd.read_csv(f'{path}\{NewSELLCEActiveContract}.csv'))
                NewSELLCEdf = NewSELLCEdf.drop(['volume','interest'] , axis =1)
                NewBuyCEdf = data_clean_backfilled(pd.read_csv(f'{path}\{NewFirstBuyCEActiveContract}.csv'))
                NewBuyCEdf = NewBuyCEdf.drop(['volume','interest'] , axis =1)
                
                NewColumns = ['datetime'] + list(CEPEdf.columns) + [f'open{NewSELLCEActiveContract}', f'high{NewSELLCEActiveContract}', f'low{NewSELLCEActiveContract}', f'close{NewSELLCEActiveContract}',\
                f'open{NewFirstBuyCEActiveContract}', f'high{NewFirstBuyCEActiveContract}',f'low{NewFirstBuyCEActiveContract}', f'close{NewFirstBuyCEActiveContract}']
                mdf1 = pd.merge_asof(CEPEdf, NewSELLCEdf, on='datetime')
                CEPEdf = pd.merge_asof(mdf1, NewBuyCEdf, on='datetime')
                CEPEdf.columns = NewColumns
                CEPEdf.set_index('datetime' , inplace=True)
                CEPEdf = CEPEdf.loc[:,~CEPEdf.columns.duplicated()]
                
                CEPEdf = CEPEdf.loc[AdjustmentTime:]
                j = 0
                
                SELLCEActiveContract = NewSELLCEActiveContract
                FirstBuyCEActiveContract = NewFirstBuyCEActiveContract
                SELLCEEntryPrice = CEPEdf.iloc[j][f'open{SELLCEActiveContract}']
                Buy1CEEntryPrice = CEPEdf.iloc[j][f'open{FirstBuyCEActiveContract}']
                print(f'Entered New SELL Contract : {SELLCEActiveContract} at Price : {SELLCEEntryPrice} and NEW BUY Contract : {FirstBuyCEActiveContract} and Price : {Buy1CEEntryPrice}')
                
                
                ## Stoploss Activation Condition:
                if (int(SELLPEActiveContract[9:14]) - int(SELLCEActiveContract[9:14])) >= -100:
                    StopLoss = (SELLCEEntryPrice + CEPEdf.iloc[j][f'open{SELLPEActiveContract}']) * 1.1
                    print(f'Stoploss Created at {CEPEdf.index[j]}')
                    
                    flag = 1
                
            
                CEEntryTime = AdjustmentTime
            
            
            
            ## PE Adjustment
            elif flag == 0 and (0.5 * CEPEdf.iloc[j][f'close{SELLCEActiveContract}']) >= CEPEdf.iloc[j][f'close{SELLPEActiveContract}']:
                AdjustedSELLPrice = 0.8 * CEPEdf.iloc[j][f'close{SELLCEActiveContract}']
                
                AdjustmentTime = CEPEdf.index[j]
                print(f'Adjustment of CE at {AdjustmentTime}')
                
                 # Updating the Orders Data frame
                 
                # Sell Order
                try:
                    ddCE = (SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : AdjustmentTime][f'high{SELLPEActiveContract}']))
                except ValueError:
                    ddCE = 0
                Orders.loc[len(Orders)] = [-1, 'PE', PEEntryTime, AdjustmentTime,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'Adj', CMP, ddCE, OrderBasketID ,SELLPEActiveContract , 1 ]
                
                # Buy Order
                try:
                    ddCE = (min(CEPEdf.loc[PEEntryTime : AdjustmentTime][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice)
                except ValueError:
                    ddCE = 0
                Orders.loc[len(Orders)] = [1, 'PE', PEEntryTime, AdjustmentTime,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'Adj', 'UP', ddCE, OrderBasketID, FirstBuyPEActiveContract, 1]
                print(f"Exited SELL Contract : {SELLPEActiveContract} at Price : {CEPEdf.iloc[j][f'close{SELLPEActiveContract}']} and NEW BUY Contract : {FirstBuyPEActiveContract} and Price : {CEPEdf.iloc[j][f'close{SELLPEActiveContract}']}")
                
                
                
                NewSELLPEActiveContract , NewSELLPEActiveContractPrice = getContractBySP_backfilled(AdjustedSELLPrice , AdjustmentTime , StrExpiry , 'PE' , path , LastThrus)
                AdjustedBUYPrice = 1/3 * NewSELLPEActiveContractPrice
                NewFirstBuyPEActiveContract , NewFirstBuyPEActiveContractPrice= getContractBySP_backfilled(AdjustedBUYPrice , AdjustmentTime , StrExpiry , 'PE' , path , LastThrus)
                
                
                NewSELLPEdf = data_clean_backfilled(pd.read_csv(f'{path}\{NewSELLPEActiveContract}.csv'))
                NewSELLPEdf = NewSELLPEdf.drop(['volume','interest'] , axis =1)
                NewBuyPEdf = data_clean_backfilled(pd.read_csv(f'{path}\{NewFirstBuyPEActiveContract}.csv'))
                NewBuyPEdf = NewBuyPEdf.drop(['volume','interest'] , axis =1)
                
                NewColumns = ['datetime'] + list(CEPEdf.columns) + [f'open{NewSELLPEActiveContract}', f'high{NewSELLPEActiveContract}', f'low{NewSELLPEActiveContract}', f'close{NewSELLPEActiveContract}',\
                f'open{NewFirstBuyPEActiveContract}', f'high{NewFirstBuyPEActiveContract}',f'low{NewFirstBuyPEActiveContract}', f'close{NewFirstBuyPEActiveContract}']
                mdf1 = pd.merge_asof(CEPEdf, NewSELLPEdf, on='datetime')
                CEPEdf = pd.merge_asof(mdf1, NewBuyPEdf, on='datetime')
                CEPEdf.columns = NewColumns
                CEPEdf.set_index('datetime' , inplace=True)
                CEPEdf = CEPEdf.loc[:,~CEPEdf.columns.duplicated()]
                
                CEPEdf = CEPEdf.loc[AdjustmentTime:]
                j = 0
                
                SELLPEActiveContract = NewSELLPEActiveContract
                FirstBuyPEActiveContract = NewFirstBuyPEActiveContract
                SELLPEEntryPrice = CEPEdf.iloc[j][f'open{SELLPEActiveContract}']
                Buy1PEEntryPrice = CEPEdf.iloc[j][f'open{FirstBuyPEActiveContract}']
                print(f'Entered New SELL Contract : {SELLPEActiveContract} at Price : {SELLPEEntryPrice} and NEW BUY Contract : {FirstBuyPEActiveContract} and Price : {Buy1PEEntryPrice}')
                
                
                ## Stoploss Activation Condition:
                if (int(SELLPEActiveContract[9:14]) - int(SELLCEActiveContract[9:14])) >= -100:
                    StopLoss = (SELLCEEntryPrice + CEPEdf.iloc[j][f'open{SELLPEActiveContract}']) * 1.1
                    print(f'Stoploss Created at {CEPEdf.index[j]}')                    
                    flag = 1


                PEEntryTime = AdjustmentTime
            
            
            
            # BreakPoint Above Exit Condition
            if TotalPnl >= Target:
                
                print(f'Target Hit at {CEPEdf.index[j]} .Closed Sell CE Contract : {SELLCEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLCEActiveContract}"]} .Closed Sell PE Contract : {SELLPEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLPEActiveContract}"]}')
                print(f'Closed Buy CE Contract : {FirstBuyCEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyCEActiveContract}"]} .Closed Buy PE Contract : {FirstBuyPEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyPEActiveContract}"]} \n\n\n\n')
                
                # Exit FIrst SELL CE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  SELLCEActiveContract,  CEEntryTime, CEPEdf.index[j] ,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, SELLCEActiveContract ,1]
                
                # Exit First SELL PE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  SELLPEActiveContract,  PEEntryTime, CEPEdf.index[j] ,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, SELLPEActiveContract,1 ]
                
    
                
                # Exit FirstBuy CE Order
                try:
                    ddCt = min(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  FirstBuyCEActiveContract,  CEEntryTime, CEPEdf.index[j] ,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, FirstBuyCEActiveContract ,1]
                
                
                # Exit FirstBuy PE Order
                try:
                    ddCt = min(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  FirstBuyPEActiveContract,  PEEntryTime, CEPEdf.index[j] ,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'BAExit', CMP, ddCt , OrderBasketID, FirstBuyPEActiveContract , 1]
                
           
                break
            
            
        
            
    
       
            
          
            
            
            # Expiry Exit Condition
            if CEPEdf.index[j] >= NextExpiry:
                print(f'Expiry Exit at {CEPEdf.index[j]} .Closed Sell CE Contract : {SELLCEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLCEActiveContract}"]} .Closed Sell PE Contract : {SELLPEActiveContract} at Price {CEPEdf.iloc[j][f"close{SELLPEActiveContract}"]}')
                print(f'Closed Buy CE Contract : {FirstBuyCEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyCEActiveContract}"]} .Closed Buy PE Contract : {FirstBuyPEActiveContract} at Price {CEPEdf.iloc[j][f"close{FirstBuyPEActiveContract}"]} \n\n\n\n')
                
                # Exit FIrst SELL CE Order
                # Exit FIrst SELL CE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  SELLCEActiveContract,  CEEntryTime, CEPEdf.index[j] ,SELLCEEntryPrice ,CEPEdf.iloc[j][f'close{SELLCEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, SELLCEActiveContract ,1]
                
                # Exit First SELL PE Order
                try:
                    ddCt = SELLPEEntryPrice - max(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'high{SELLPEActiveContract}']) 
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [-1 ,  SELLPEActiveContract,  PEEntryTime, CEPEdf.index[j] ,SELLPEEntryPrice ,CEPEdf.iloc[j][f'close{SELLPEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, SELLPEActiveContract,1 ]
                
               
                
                # Exit FirstBuy CE Order
                try:
                    ddCt = min(CEPEdf.loc[CEEntryTime : CEPEdf.index[j]][f'low{FirstBuyCEActiveContract}']) - Buy1CEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  FirstBuyCEActiveContract,  CEEntryTime, CEPEdf.index[j] ,Buy1CEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyCEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, FirstBuyCEActiveContract ,1]
                
                
                # Exit FirstBuy PE Order
                try:
                    ddCt = min(CEPEdf.loc[PEEntryTime : CEPEdf.index[j]][f'low{FirstBuyPEActiveContract}']) - Buy1PEEntryPrice
                except ValueError:
                    ddCt = 0
                Orders.loc[len(Orders)] = [1 ,  FirstBuyPEActiveContract,  PEEntryTime, CEPEdf.index[j] ,Buy1PEEntryPrice ,CEPEdf.iloc[j][f'close{FirstBuyPEActiveContract}'], 'Exp', CMP, ddCt , OrderBasketID, FirstBuyPEActiveContract , 1]
              
               
                break
               
            
            
        
        

    
            j+=1
                
        # Break For 1 Trade
        #break
                    
                

            

2020-01-02 10:00:00
09JAN20
False
BANKNIFTY3250009JAN20CE
Entry at 2020-01-02 10:00:00 at a SP of 32200
Enter First Sell Series CE Contract : BANKNIFTY3250009JAN20CE Price : 108.55 . PE Contract : BANKNIFTY3190009JAN20PE Price : 94.9
Enter First Buy Series CE Contract : BANKNIFTY3290009JAN20CE Price : 28.8 . PE Contract : BANKNIFTY3150009JAN20PE Price : 30.5
Adjustment of CE at 2020-01-02 12:14:00
Exited SELL Contract : BANKNIFTY3190009JAN20PE at Price : 71.85 and NEW BUY Contract : BANKNIFTY3150009JAN20PE and Price : 71.85
Entered New SELL Contract : BANKNIFTY3210009JAN20PE at Price : 126.75 and NEW BUY Contract : BANKNIFTY3170009JAN20PE and Price : 42.2
Adjustment of CE at 2020-01-02 14:23:00
Exited SELL Contract : BANKNIFTY3210009JAN20PE at Price : 92.9 and NEW BUY Contract : BANKNIFTY3170009JAN20PE and Price : 92.9
Entered New SELL Contract : BANKNIFTY3230009JAN20PE at Price : 159.4 and NEW BUY Contract : BANKNIFTY3190009JAN20PE and Price : 56.4
Adjustment of CE at 2020-01-03 09:30

In [10]:
FirstBuyCEActiveContract

'BANKNIFTY34900DEC21CE'

In [6]:
(tdayTime + timedelta(days=1)).date() in HolidayDateList

False

In [3]:
Orders

,entry,type,entry_time,exittime,entry_value,exitvalue,exit_type,SP,dd,BasketID,ContractName,LS
0,-1,PE,2020-01-02 10:00:00,2020-01-02 12:14:00,94.90,71.85,Adj,32200,-6.35,1,BANKNIFTY3190009JAN20PE,1
1,1,PE,2020-01-02 10:00:00,2020-01-02 12:14:00,30.50,23.45,Adj,UP,-7.25,1,BANKNIFTY3150009JAN20PE,1
2,-1,PE,2020-01-02 12:14:00,2020-01-02 14:23:00,126.75,92.90,Adj,32200,-13.25,1,BANKNIFTY3210009JAN20PE,1
3,1,PE,2020-01-02 12:14:00,2020-01-02 14:23:00,42.20,31.25,Adj,UP,-10.95,1,BANKNIFTY3170009JAN20PE,1
4,-1,CE,2020-01-02 10:00:00,2020-01-03 09:30:00,108.55,119.00,Adj,32200,-108.45,1,BANKNIFTY3250009JAN20CE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1643,1,CE,2021-12-24 09:42:00,2021-12-27 09:15:00,75.20,23.10,Adj,UP,-53.15,101,BANKNIFTY35900DEC21CE,1
1644,-1,BANKNIFTY34200DEC21CE,2021-12-27 09:15:00,2021-12-27 09:16:00,627.15,490.35,SL,35200,-431.25,101,BANKNIFTY34200DEC21CE,1
1645,-1,BANKNIFTY34800DEC21PE,2021-12-24 14:15:00,2021-12-27 09:16:00,249.15,607.55,SL,35200,-431.25,101,BANKNIFTY34800DEC21PE,1
1646,1,BANKNIFTY34900DEC21CE,2021-12-27 09:15:00,2021-12-27 09:16:00,344.00,158.55,SL,35200,-202.00,101,BANKNIFTY34900DEC21CE,1


In [54]:
EntryWeekDay

2

In [50]:
checkLastThrusday(tdayTime,  NextExpiry)

True

In [51]:
tdayTime

datetime.datetime(2020, 4, 23, 10, 0)

In [44]:
tdayTime

datetime.datetime(2020, 4, 15, 10, 0)

In [45]:
StrExpiry

'23APR20'

BANKNIFTY2350019MAR20PE 199.0
BANKNIFTY2400019MAR20PE 187.60000000000002


In [27]:
SELLPEActiveContract

'BANKNIFTY2400019MAR20PE'

In [26]:
SELLPEActiveContractPrice

287.6

In [46]:
data.loc[:tdayTime]

,open,high,low,close,volume
datetime,,,,,
2019-11-18 09:15:00,31036.80,31138.75,31036.25,31133.70,0
2019-11-18 09:16:00,31123.95,31143.10,31117.50,31140.65,0
2019-11-18 09:17:00,31139.90,31152.50,31132.45,31141.10,0
2019-11-18 09:18:00,31140.75,31161.25,31116.80,31149.25,0
2019-11-18 09:19:00,31144.15,31158.70,31134.60,31140.40,0
...,...,...,...,...,...
2020-04-15 09:56:00,19969.35,19969.35,19906.05,19915.35,0
2020-04-15 09:57:00,19915.45,19938.55,19911.15,19923.70,0
2020-04-15 09:58:00,19922.55,19954.90,19920.30,19950.40,0


In [20]:
SELLPEActiveContract , SELLPEActiveContractPrice = getContractBySP_backfilled(Price, tdayTime, StrExpiry,'PE',path, LastThrus)

C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\IndianOptionBackFilled\2020_new\BANKNIFTY2350019MAR20PE.csv
BANKNIFTY2350019MAR20PE 199.0
C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\IndianOptionBackFilled\2020_new\BANKNIFTY2450019MAR20PE.csv
C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\IndianOptionBackFilled\2020_new\BANKNIFTY2480019MAR20PE.csv
C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\IndianOptionBackFilled\2020_new\BANKNIFTY2500019MAR20PE.csv
C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\IndianOptionBackFilled\2020_new\BANKNIFTY2530019MAR20PE.csv
C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\IndianOptionBackFilled\2020_new\BANKNIFTY2550019MAR20PE.csv
C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\IndianOptionBackFilled\2020_new\BANKNIFTY2570019MAR20PE.csv
C:\Users\soura\Documents\PythonTrading\BacktestResult\OptionSrategy\Indian

In [12]:
SELLPEActiveContract

'BANKNIFTY2400019MAR20PE'

In [6]:
list(CEPEdf.columns) + [f'open{NewSELLCEActiveContract}', f'high{NewSELLCEActiveContract}', f'low{NewSELLCEActiveContract}', f'close{NewSELLCEActiveContract}',\
f'open{NewFirstBuyCEActiveContract}', f'high{NewFirstBuyCEActiveContract}',f'low{NewFirstBuyCEActiveContract}', f'close{NewFirstBuyCEActiveContract}']

['datetime',
 'openBANKNIFTY3310014JAN21CE',
 'highBANKNIFTY3310014JAN21CE',
 'lowBANKNIFTY3310014JAN21CE',
 'closeBANKNIFTY3310014JAN21CE',
 'openBANKNIFTY3100014JAN21PE',
 'highBANKNIFTY3100014JAN21PE',
 'lowBANKNIFTY3100014JAN21PE',
 'closeBANKNIFTY3100014JAN21PE',
 'openBANKNIFTY3380014JAN21CE',
 'highBANKNIFTY3380014JAN21CE',
 'lowBANKNIFTY3380014JAN21CE',
 'closeBANKNIFTY3380014JAN21CE',
 'openBANKNIFTY3010014JAN21PE',
 'highBANKNIFTY3010014JAN21PE',
 'lowBANKNIFTY3010014JAN21PE',
 'closeBANKNIFTY3010014JAN21PE',
 'openSpot',
 'highSpot',
 'lowSpot',
 'closeSpot',
 'volumeSpot',
 'open_x',
 'high_x',
 'low_x',
 'close_x',
 'open_y',
 'high_y',
 'low_y',
 'close_y',
 'openBANKNIFTY3290014JAN21CE',
 'highBANKNIFTY3290014JAN21CE',
 'lowBANKNIFTY3290014JAN21CE',
 'closeBANKNIFTY3290014JAN21CE',
 'openBANKNIFTY3340014JAN21CE',
 'highBANKNIFTY3340014JAN21CE',
 'lowBANKNIFTY3340014JAN21CE',
 'closeBANKNIFTY3340014JAN21CE']

In [8]:
CEPEdf

,datetime,openBANKNIFTY3310014JAN21CE,highBANKNIFTY3310014JAN21CE,lowBANKNIFTY3310014JAN21CE,closeBANKNIFTY3310014JAN21CE,openBANKNIFTY3100014JAN21PE,highBANKNIFTY3100014JAN21PE,lowBANKNIFTY3100014JAN21PE,closeBANKNIFTY3100014JAN21PE,openBANKNIFTY3380014JAN21CE,...,closeSpot,volumeSpot,open_x,high_x,low_x,close_x,open_y,high_y,low_y,close_y
0,2021-01-07 10:00:00,99.40,100.00,99.40,100.00,105.05,105.10,101.50,101.50,33.35,...,32051.20,0,138.95,138.95,138.95,138.95,59.95,59.95,59.55,59.55
1,2021-01-07 10:01:00,99.50,100.60,99.50,99.85,101.85,103.00,101.60,101.85,33.50,...,32052.25,0,137.80,137.80,137.80,137.80,59.70,60.15,59.70,60.15
2,2021-01-07 10:03:00,99.85,99.85,99.60,99.60,103.00,103.00,100.15,101.80,32.75,...,32044.20,0,138.00,138.00,136.60,136.60,60.20,60.20,59.70,59.70
3,2021-01-07 10:04:00,98.00,98.00,95.70,96.10,101.85,105.00,101.35,103.15,32.55,...,32016.50,0,135.00,135.40,135.00,135.40,59.05,59.25,57.65,57.65
4,2021-01-07 10:06:00,93.00,94.50,92.85,94.00,105.40,108.00,105.00,106.35,31.00,...,31994.75,0,130.00,131.40,129.15,129.75,57.50,57.75,56.00,56.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,2021-01-14 15:25:00,0.15,0.20,0.10,0.15,0.10,0.20,0.05,0.10,0.10,...,32553.50,0,0.10,0.15,0.05,0.05,0.05,0.15,0.05,0.15
2087,2021-01-14 15:26:00,0.20,0.20,0.10,0.10,0.05,0.15,0.05,0.10,0.10,...,32555.15,0,0.05,0.15,0.05,0.05,0.05,0.15,0.05,0.15
2088,2021-01-14 15:27:00,0.15,0.15,0.10,0.15,0.10,0.15,0.05,0.05,0.15,...,32566.75,0,0.10,0.10,0.05,0.05,0.10,0.10,0.05,0.05
2089,2021-01-14 15:28:00,0.15,0.15,0.10,0.10,0.10,0.15,0.05,0.05,0.10,...,32560.70,0,0.05,0.05,0.05,0.05,0.15,0.15,0.05,0.05


## Performance Metrics

In [17]:
class backtesting_final_per_pos:
    
    def __init__(self, ticker, df,lotsize,commstr, filename,size):
        self.filename = filename


        
        self.symbol = ticker
        self.df = df
        self.per = lotsize
        self.str = commstr   # Yes for commission and no without commission
        #self.pip , self.qty = self.DataAcessPipQuantityDropBox(self.symbol)
        self.size = size
        self.pip , self.qty = 1,15
        self.trades = self.df.copy()
        #self.trades.to_csv('cc.csv')
        #self.trades = self.trades.dropna()
        #self.trades = self.trades.sort_values(by='entry_time')
        self.td = self.analysis(self.trades)
        self.longwinper = self.long_win()
        self.sharpe = self.sharperatio()
        #self.backtest_reports_local()
        self.run()
        

    def run(self):
        fig1 = self.backtest_reports_local()
        trades = self.finalreport()
        with open(f'{self.filename}_{self.symbol}_Performance.txt', 'r') as txt_file:
            txt_content = txt_file.read()
            
        html_content = f"<pre>{txt_content}</pre>"
        #fig2 = self.RandomTradesShuffleSimultion()
        #fig3 = self.RandomTradesRemovalSimulation(0.2)
        fig4 = self.HoldingTimeScatterPlot()
        fig5 , fig6 = self.Hour_Day_Analysis()
        #fig7 = self.Monthlyreturns()
        trades_html = trades.to_html(index=False)

        with open(f'{self.filename}_{self.symbol}_Performance.html', 'a') as f:
            f.write("<h2>Performance of The Strategy</h2>\n")
            f.write(html_content)
            f.write("<h2>Metrics of The Strategy</h2>\n")
            f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn'))
            #f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
            #f.write(fig3.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write(fig4.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write(fig5.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write(fig6.to_html(full_html=False, include_plotlyjs='cdn'))
            f.write("<h2>Trades Data</h2>\n")  # Add a heading for the CSV data (optional)
            f.write(trades_html)
            #f.write(fig7.to_html(full_html=False, include_plotlyjs='cdn'))
    
    def DataAcessPipQuantityDropBox(self,symbol ):
        print('step2')
        # Access Pip and Quantity
       
        df_cal = pd.read_csv('https://www.dropbox.com/s/r4f8cmb9jkv6g6n/PipVAlue.csv?dl=1',encoding = 'utf-8')
        import ast
        for i in range(len(df_cal)):
            if df_cal.iloc[i]['Links'].find('nan') < 0:
                df_cal['Links'][i] = ast.literal_eval(df_cal['Links'][i].replace('[','').replace(']',''))

        PIP = df_cal['PipValue'].where(df_cal['INS_Name'] == symbol).dropna()
        PIP = PIP[PIP.index[0]]

        Quant = df_cal['Quantity'].where(df_cal['INS_Name'] == symbol).dropna()
        Quant = Quant[Quant.index[0]]

        return PIP , Quant

        
        
        
    
       
    def analysis(self , trades):
        
        #print(trades)
        
        #print(trades)
        trades["returns"] = None
        trades['returns_abs'] = None
        trades['returns_dd'] = None
        trades['per_dd'] = None
        trades['lot_size'] = 1
        trades['pips'] = None
        trades['commision'] = 0
        trades['MarginRequired'] = 0
        trades.reset_index(drop=True,inplace=True)
        #print(trades)
        #trades['returns'].values[0] = 100
        #print(trades['returns'].values[0])
       
        
        for i in range(len(trades)):
            #print(trades.iloc[i])
            #commission 
            if self.str == 'Yes':
                commision = 2
                MarginRequired = trades.iloc[i]["entry_value"] * self.per
                if i == 0:
                    
                    trades['lot_size'].values[i] = self.per
                    if(trades['entry'].values[i] == 1):
                        trades.loc[i,"returns"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])* self.pip * trades.iloc[i]['lot_size'] * self.qty  - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])
                        
                    else:
                        trades.loc[i,"returns"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])* self.pip * trades.iloc[i]['lot_size'] * self.qty   - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])
                    trades.loc[i,'returns_abs'] = self.size + trades.iloc[i]["returns"]
                    trades.loc[i , 'commision'] = commision
                    trades.loc[i , 'MarginRequired'] = MarginRequired
                else:
                    trades['lot_size'].values[i] = self.per
                    if(trades['entry'].values[i] == 1):
                        trades.loc[i,"returns"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])* self.pip * trades.iloc[i]['lot_size'] * self.qty  - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])
                    else:
                        trades.loc[i,"returns"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])* self.pip * trades.iloc[i]['lot_size'] * self.qty  - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])
                    trades.loc[i,'returns_abs'] = trades.iloc[i-1]['returns_abs'] + trades.iloc[i]['returns']
                    trades.loc[i , 'commision'] = commision
                    trades.loc[i , 'MarginRequired'] = MarginRequired

            
           #without comissiom
            elif self.str == 'No':
                #commision = calculate_stock_holding_commision(trades.iloc[i]["entry_value"] , trades.iloc[i]["exitvalue"], self.per)
                commision = 60
                MarginRequired = trades.iloc[i]["entry_value"] * self.per * self.qty
                if i == 0:
                    
                    
                    trades['lot_size'].values[i] = self.per
                    if(trades['entry'].values[i] == 1):
                        trades.loc[i,"returns"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])* self.pip * trades.iloc[i]['lot_size'] * self.qty  - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])
                        
                    else:
                        trades.loc[i,"returns"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])* self.pip * trades.iloc[i]['lot_size'] * self.qty   - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])
                    trades.loc[i,'returns_abs'] = self.size + trades.iloc[i]["returns"]
                    trades.loc[i , 'commision'] = commision
                    trades.loc[i , 'MarginRequired'] = MarginRequired
                else:
                    trades['lot_size'].values[i] = self.per
                    if(trades['entry'].values[i] == 1):
                        trades.loc[i,"returns"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])* self.pip * trades.iloc[i]['lot_size'] * self.qty  - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["exitvalue"] - trades.iloc[i]["entry_value"])
                    else:
                        trades.loc[i,"returns"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])* self.pip * trades.iloc[i]['lot_size'] * self.qty  - commision
                        trades.loc[i,"pips"]=(trades.iloc[i]["entry_value"] - trades.iloc[i]["exitvalue"])
                    trades.loc[i,'returns_abs'] = trades.iloc[i-1]['returns_abs'] + trades.iloc[i]['returns']
                    trades.loc[i , 'commision'] = commision
                    trades.loc[i , 'MarginRequired'] = MarginRequired
        
        


        

        trades.loc[0,'returns_dd'] = abs(trades.iloc[0]["dd"]* self.pip * trades.iloc[0]['lot_size'] *self.qty)
        for i in range(1,len(trades)):

            trades.loc[i,'returns_dd'] = abs(trades.iloc[i]['dd']* self.pip * trades.iloc[i]['lot_size'] *self.qty)

        trades.loc[0,'per_dd'] = (trades.iloc[0]["returns_dd"]/self.size )*100
        for i in range(1,len(trades)):

            trades.loc[i,'per_dd'] = (trades.iloc[i]["returns_dd"]/trades.iloc[i-1]["returns_abs"])*100 


        trades['dd'] = trades['dd']*self.qty * trades['lot_size']
        #trades['Duration'] = trades.iloc[:]['exittime'] - trades.iloc[:]['entry_time']
    
        return trades
        


    def long_win(self):
    
        long_df = self.td[self.td['entry'] == 1]
        long_df
        #print(long_df[long_df.returns>0]['returns'].count()/long_df['entry'].count())
        return(round((long_df[long_df.returns>0]['returns'].count()/long_df['entry'].count()) * 100,2))
    
    
    def finalreport(self):
        
        trades = self.td[['type','entry_time','exittime','entry_value','exitvalue','entry','lot_size','pips','returns','returns_abs','exit_type','SP','dd','per_dd','ContractName','BasketID','commision','MarginRequired']].copy()

        trades.rename(columns = {'entry_value':'Entryvalue'}, inplace = True)
        trades.rename(columns = {'returns':'PnL'}, inplace = True)
        trades.rename(columns = {'returns_abs':'Equity'}, inplace = True)

        trades['entry'] = trades['entry'].replace([1,-1],['Long','Short'])
        #trades['dd'] = trades['dd'] * self.qty
        trades['Duration'] = trades['exittime'] - trades['entry_time']
        trades.to_csv(f'{self.filename}_{self.symbol}_trades.csv',index = False)

        return trades#.to_string(index=False)
    
    # Assume an average annual risk-free rate is 1%
    def sharperatio(self):
        risk_free_rate = 0.05/252

        sharpe_ratio = (np.mean(self.td.returns) -
                           (risk_free_rate))/np.std(self.td.returns)
        return sharpe_ratio
    


    def backtest_reports_local(self):
        df_summary = self.td.copy()

        basketReturns = pd.DataFrame(columns=['returns' , 'Orders'])
        for i in df_summary.BasketID.unique():
            Bdf = df_summary[df_summary.BasketID == i]
            
            basketReturns.loc[i, 'returns'] = Bdf.returns.sum()
            basketReturns.loc[i, 'Orders'] = Bdf.returns.count() -2
            exitType = Bdf.exit_type.unique()
            for j in exitType:
                if j == 'DEAdj':
                    continue
                else:
                    exitValue = j
                    
            
            basketReturns.loc[i, 'ExitType'] = exitValue
            
            #print('\n')
            
            #print(Bdf.iloc[-1])

        BasketWinPer = (len(basketReturns[basketReturns.returns >0]) / len(basketReturns)) * 100
        BasketAvgWin = basketReturns[basketReturns.returns >0].returns.mean()
        BasketAvgLoss = basketReturns[basketReturns.returns <0].returns.mean()
        AvgAdjustetment = basketReturns.Orders.mean()
        MaxAdjustment = basketReturns.Orders.max()

        TPExitwinPer = (len(basketReturns[basketReturns.ExitType == 'TP']) / len(basketReturns)) * 100
        BBExitwinPer = (len(basketReturns[basketReturns.ExitType == 'BBExit']) / len(basketReturns)) * 100
        BAExitwinPer = (len(basketReturns[basketReturns.ExitType == 'BAExit']) / len(basketReturns)) * 100
        EXPExitwinPer = (len(basketReturns[basketReturns.ExitType == 'Exp']) / len(basketReturns)) * 100
        df_summary.set_index('entry_time', inplace=True)
  
        
        #print(df_summary)
        print("Number of trades: ", df_summary['entry'].count())
        print(f"Total No of Basket : {len(basketReturns)}")
        #print("Number of positions: ", len(df_summary[df_summary['exittype'] == 'SL']))
        #print(f'Total No of Basket : {TotalBakset}')
        print("Win percentage: ", round((df_summary[df_summary.returns>0]['returns'].count()/df_summary['entry'].count()) * 100,2))
        print(f'Win Percentage of Bakset {BasketWinPer} ')
        print(f"TP Exit  Percentage : {TPExitwinPer}")
        print(f"BreakPoint Above Exit Percentage : {BAExitwinPer}")
        print(f"BreakPoint Below Exit Percentage : {BBExitwinPer}")
        print(f"Expiry Exit Percentage : {EXPExitwinPer}")
        #print(f"Basket Win Percentage : {round( (winTradesCount/(winTradesCount+lossTradesCount)) * 100,2  ) }")
        print("Net PnL: ", round(df_summary['returns'].sum(),2))
        #print("Average PnL: ", round(df_summary['returns'].mean(),2))
        print("No of winning trades: ", df_summary[df_summary.returns>0]['returns'].count())
        #print(f'No of Winning Basket : {winTradesCount}')
        print("No of losing trades:", df_summary[df_summary.returns<0]['returns'].count())
        #print(f"No of Losing Basket : {lossTradesCount}")

        print("Win % of Long trades: ", self.longwinper)
        
        short_df = df_summary[df_summary['entry'] == -1]
        #print("Number of short trades: ", df_summary[df_summary.entry == -1]['entry'].count())
        print("Win % of Short trades: ",round((short_df[short_df.returns>0]['returns'].count()/short_df['entry'].count()) * 100,2))

        print("Final Balance($): ", round(df_summary['returns_abs'].values[-1],2))
        print("Average_gain(pips): ", round(df_summary[df_summary.pips>0]['pips'].mean(),2))
        print("Average_loss(pips): ", round(df_summary[df_summary.pips<0]['pips'].mean(),2))
        print("Average_gain($): ", round(df_summary[df_summary.returns>0]['returns'].mean(),2))
        print("Average_loss($): ", round(df_summary[df_summary.returns<0]['returns'].mean(),2))
        print(f"Avergain Gain of a Basket : {BasketAvgWin} ")
        print(f"Avergain Loss of a Basket : {BasketAvgLoss} ")
        
        print(f"Avg Adjustment of the Basket : {AvgAdjustetment}")
        print(f"Max Adjustment of the Basket : {MaxAdjustment}")
        print("Profit factor: ", abs(round((df_summary[df_summary.returns>0]['returns'].sum())/(df_summary[df_summary.returns<0]['returns'].sum()),2)))
        print("Sharpe ratio: ", round(self.sharpe,2))
        with open(f'{self.filename}_{self.symbol}_Performance.txt', 'a') as f:
            f.write(f"Number of trades : {df_summary['entry'].count()}\n")
            f.write(f"Total No of Basket : {len(basketReturns)}\n")
            #f.write(f'Total No of Basket : {TotalBakset}\n')
            f.write(f"Win percentage : { round((df_summary[df_summary.returns>0]['returns'].count()/df_summary['entry'].count()) * 100,2)}\n")
            f.write(f'Win Percentage of Bakset {BasketWinPer} \n')
            f.write(f"TP Exit  Percentage : {TPExitwinPer} \n")
            f.write(f"BreakPoint Above Exit Percentage : {BAExitwinPer} \n")
            f.write(f"BreakPoint Below Exit Percentage : {BBExitwinPer} \n")
            f.write(f"Expiry Exit Percentage : {EXPExitwinPer} \n")
            #f.write(f"Basket Win Percentage : {round( (winTradesCount/(winTradesCount+lossTradesCount)) * 100,2  ) }\n")
            f.write(f"Net PnL : {round(df_summary['returns'].sum(),2)}\n")
            f.write(f"No of winning trades : {df_summary[df_summary.returns>0]['returns'].count()}\n")
            #f.write(f'No of Winning Basket : {winTradesCount}\n')
            f.write(f"No of losing trades : {df_summary[df_summary.returns<0]['returns'].count()}\n")
            #f.write(f"No of Losing Basket : {lossTradesCount}\n")
            f.write(f"Win % of Long trades : {self.longwinper}\n")
            f.write(f"Win % of Short trades : {round((short_df[short_df.returns>0]['returns'].count()/short_df['entry'].count()) * 100,2)}\n")
            f.write(f"Final Balance($) : {round(df_summary['returns_abs'].values[-1],2)}\n")
            f.write(f"Average_gain(pips) : { round(df_summary[df_summary.returns>0]['returns'].mean(),2)}\n")
            f.write(f"Average_loss(pip) : { round(df_summary[df_summary.returns<0]['returns'].mean(),2)}\n")
            f.write(f"Average_gain($) : {round(df_summary[df_summary.pips<0]['pips'].mean(),2)}\n")
            f.write(f"Average_loss($) : {round(df_summary[df_summary.returns<0]['returns'].mean(),2)}\n")
            f.write(f"Avergain Gain of a Basket : {BasketAvgWin} \n")
            f.write(f"Avergain Loss of a Basket : {BasketAvgLoss} \n")
            #f.write(f"Avg Adjustment of the Basket : {AvgAdjustetment}\n")
            f.write(f"Max Adjustment of the Basket : {MaxAdjustment}\n")
            f.write(f"Profit factor : {abs(round((df_summary[df_summary.returns>0]['returns'].sum())/(df_summary[df_summary.returns<0]['returns'].sum()),2))}\n")
            f.write(f"Sharpe ratio : {round(self.sharpe,2)}\n")
        
        fig1 = px.line(df_summary , y='returns_abs',x = df_summary['exittime'] , labels=['PnL'] , title= 'Equity Graph' ,template='plotly_dark' )
        fig1.show()
        return fig1
    def RandomTradesRemovalSimulation(self , frac):
       
        trades = self.trades.copy()
        #print(trades)
        #trades.reset_index(drop = True, inplace = True)
        #print(trades)
        randomDf = pd.DataFrame()
        for i in range(int(len(trades)/2)):
            df = trades.sample(frac = 1-frac )
            df.reset_index(drop = True, inplace = True)
            #print(df)
            td = self.analysis(df)
            #print(td)
            # if i == 0:
            #     randomDf = td
            # else :

            #     randomDf.append(td)

            # randomDf[f'R_{i}'] = td['returns']
            randomDf[i] = td['returns_abs'].to_list()

        fig1 = px.line(randomDf , y=list(randomDf.columns) , title='RandomTradesRemovalSimulation' ,template='plotly_dark' )
        #fig1.show()
        #return random
        # Df
        return fig1

    def RandomTradesShuffleSimultion(self):
        trades = self.trades.copy()
        all_td = []
        randomDf = pd.DataFrame()
        randomDf['original'] = self.analysis(trades)['returns_abs'].to_list()
        for i in range(int(len(trades)/2)):
            df = trades.sample(frac = 1)
            df.reset_index(drop = True, inplace = True)
            td = self.analysis(df)
            all_td.append(td)

            randomDf[f'R_{i}'] = td['returns_abs'].to_list()
            #randomDf[i] = td['returns_abs'].to_list()
        fig1 = px.line(randomDf , y=list(randomDf.columns) , title='RandomTradesShuffleSimulation' ,template='plotly_dark' )
        #fig1.show()
        return fig1
        #return randomDf

    def Hour_Day_Analysis(self):

        trades = self.trades.copy()
        trades['Hour'] = list(i.hour for i in trades['entry_time'])
        trades['Day'] = list(i.weekday() for i in trades['entry_time'])
        

        df_win = trades.query('returns > 0')
        HourWin = df_win.groupby('Hour').returns.agg({'count'})['count'].to_list()
        Hours = df_win.groupby('Hour').returns.agg({'count'}).index.to_list()
        DayWin = df_win.groupby('Day').returns.agg({'count'})['count'].to_list()
        Day = df_win.groupby('Day').returns.agg({'count'}).index.to_list()


        df_loose = trades.query('returns < 0')
        HourLoose = df_loose.groupby('Hour').returns.agg({'count'})['count'].to_list()
        DayLoose = df_loose.groupby('Day').returns.agg({'count'})['count'].to_list()

        HourDf = pd.DataFrame( data=list(zip(HourWin, HourLoose)), columns=['Win','Loss'])

        fig1 = px.bar(HourDf ,title='HourAnalysis' ,template='plotly_dark' )
        #fig1.show()

        DayLoose = df_loose.groupby('Day').returns.agg({'count'})['count'].to_list()

        DayDf = pd.DataFrame( data=list(zip(DayWin, DayLoose)), columns=['Win','Loss'])
        fig2 = px.bar(DayDf  , title='DayAnalysis' ,template='plotly_dark' )
        #fig2.show()
        return fig1,fig2

        #return HourDf

    def HoldingTimeScatterPlot(self ):
        import plotly.express as px
        trades = self.finalreport()
        hdf = trades[['Duration' , 'PnL','exit_type']].copy()
        
        hdf.reset_index(inplace=True,drop=True)
        hdf['Duration'] = list((i.total_seconds()/60) for i in hdf['Duration'])
        fig = px.scatter(hdf , y = 'PnL',x='Duration' , color='exit_type',width = 1000,height = 500)
        fig.update_traces(marker_size=8)
        #fig.show()

        return fig
    
    def Monthlyreturns(self):
        td = self.trades.copy()
        td.set_index('entry_time',inplace = True)
        mrdf = pd.DataFrame(td.returns.groupby(pd.Grouper(freq="M")).sum())

        df20 = mrdf[mrdf.index.year == 2020]
        df21 = mrdf[mrdf.index.year == 2021]
        df22 = mrdf[mrdf.index.year == 2022]
        
        
        fdf = pd.DataFrame()
        fdf['2020'] = list(df20.returns)
        fdf['2021'] = list(df21.returns)
        fdf['2022'] = list(df22.returns)

        fig1 = px.line(fdf, title='Monthly Returns' ,template='plotly_dark' )

        return fig1
        





        







        
         

In [18]:

filename = 'IronCondorBaseNonBackfilled'
res_pos1 = backtesting_final_per_pos('', Orders.sort_values(by='exittime').reset_index(drop=True),1,'No', filename,500000)


Number of trades:  1290
Total No of Basket : 76
Win percentage:  46.74
Win Percentage of Bakset 47.368421052631575 
TP Exit  Percentage : 0.0
BreakPoint Above Exit Percentage : 9.210526315789473
BreakPoint Below Exit Percentage : 0.0
Expiry Exit Percentage : 14.473684210526317
Net PnL:  -935.25
No of winning trades:  603
No of losing trades: 687
Win % of Long trades:  12.09
Win % of Short trades:  81.4
Final Balance($):  499064.75
Average_gain(pips):  68.95
Average_loss(pips):  -57.33
Average_gain($):  1012.86
Average_loss($):  -890.38
Avergain Gain of a Basket : 1730.3958333333333 
Avergain Loss of a Basket : -1580.7374999999997 
Avg Adjustment of the Basket : 14.973684210526315
Max Adjustment of the Basket : 38
Profit factor:  1.0
Sharpe ratio:  -0.0


In [19]:
Orders

,entry,type,entry_time,exittime,entry_value,exitvalue,exit_type,SP,dd,BasketID,ContractName,LS
0,-1,PE,2020-01-02 10:00:00,2020-01-02 12:14:00,94.90,71.85,Adj,32200,-6.35,1,BANKNIFTY3190009JAN20PE,1
1,1,PE,2020-01-02 10:00:00,2020-01-02 12:14:00,30.50,23.45,Adj,UP,-7.25,1,BANKNIFTY3150009JAN20PE,1
2,-1,PE,2020-01-02 12:14:00,2020-01-02 14:23:00,126.75,92.90,Adj,32200,-13.25,1,BANKNIFTY3210009JAN20PE,1
3,1,PE,2020-01-02 12:14:00,2020-01-02 14:23:00,42.20,31.25,Adj,UP,-10.95,1,BANKNIFTY3170009JAN20PE,1
4,-1,CE,2020-01-02 10:00:00,2020-01-03 09:30:00,108.55,119.00,Adj,32200,-108.45,1,BANKNIFTY3250009JAN20CE,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1285,1,CE,2021-12-17 12:09:00,2021-12-17 15:01:00,108.40,85.35,Adj,UP,-27.20,100,BANKNIFTY3670023DEC21CE,1
1286,-1,CE,2021-12-17 15:01:00,2021-12-20 09:15:00,437.00,180.25,SL,36900,-791.50,100,BANKNIFTY3560023DEC21CE,1
1287,-1,PE,2021-12-16 10:00:00,2021-12-20 09:15:00,100.55,866.05,SL,36900,-791.50,100,BANKNIFTY3580023DEC21PE,1
1288,1,CE,2021-12-17 15:01:00,2021-12-20 09:15:00,142.85,44.00,SL,36900,-110.45,100,BANKNIFTY3640023DEC21CE,1


In [14]:
Orders

,entry,type,entry_time,exittime,entry_value,exitvalue,exit_type,SP,dd,BasketID,ContractName,LS
0,-1,BANKNIFTY35700JUL21CE,2021-07-22 10:00:00,2021-07-28 15:18:00,105.15,5.95,BAExit,34900,-195.10,1,BANKNIFTY35700JUL21CE,2
1,-1,BANKNIFTY34100JUL21PE,2021-07-22 10:00:00,2021-07-28 15:18:00,93.00,30.15,BAExit,34900,-195.10,1,BANKNIFTY34100JUL21PE,2
2,1,BANKNIFTY36400JUL21CE,2021-07-22 10:00:00,2021-07-28 15:18:00,34.90,3.05,BAExit,34900,-34.80,1,BANKNIFTY36400JUL21CE,1
3,1,BANKNIFTY33300JUL21PE,2021-07-22 10:00:00,2021-07-28 15:18:00,34.20,4.30,BAExit,34900,-30.40,1,BANKNIFTY33300JUL21PE,1
4,-1,BANKNIFTY3550005AUG21CE,2021-07-29 10:00:00,2021-08-03 09:35:00,92.40,21.00,BAExit,34600,-45.95,2,BANKNIFTY3550005AUG21CE,2
...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,BANKNIFTY3450023DEC21PE,2021-12-15 10:00:00,2021-12-23 10:00:00,34.15,7.70,Exp,36900,-27.70,22,BANKNIFTY3450023DEC21PE,1
88,-1,BANKNIFTY36100DEC21CE,2021-12-22 10:00:00,2021-12-29 09:15:00,90.25,18.70,BAExit,34900,-62.15,23,BANKNIFTY36100DEC21CE,2
89,-1,BANKNIFTY33600DEC21PE,2021-12-22 10:00:00,2021-12-29 09:15:00,110.05,17.00,BAExit,34900,-62.15,23,BANKNIFTY33600DEC21PE,2
90,1,BANKNIFTY37000DEC21CE,2021-12-22 10:00:00,2021-12-29 09:15:00,31.65,4.50,BAExit,34900,-27.65,23,BANKNIFTY37000DEC21CE,1
